#### Teste End to End Import de Funções e Libs

###### Função de filtro para selecionar um aprovado e um recusado de cada etapa

In [1]:
import pandas as pd
def arquivoExemplos(massa,listaResultado,tipoCenario):
    # Inicia os ids com -1, caso não encontre cenários aprovados e negados
    idSolicitacaoAprovado = -1
    idSolicitacaoNegado = -1
    # Busca um id aprovado
    for payload in listaResultado:
        if payload['payloadHomol']['saidas']['statusDecisao'] == 'APROVADO':
            idSolicitacaoAprovado = payload['idSolicitacao']
            break
    # Busca um id negado
    for payload in listaResultado:
        if payload['payloadHomol']['saidas']['statusDecisao'] == 'NEGADO':
            idSolicitacaoNegado = payload['idSolicitacao']
            break

    if idSolicitacaoAprovado == -1 or idSolicitacaoNegado == -1:
        print('Não foi possível encontrar um cenário aprovado e um cenário negado')
    else:
        # partindo do principio que o idSolicitacao no json é gerado a partir do indice da massa no sherlock
        aprovado = massa.iloc[[idSolicitacaoAprovado]].copy()
        aprovado['Debug - ResultadoEsperado'] = 'APROVADO'

        recusado = massa.iloc[[idSolicitacaoNegado]].copy()
        recusado['Debug - ResultadoEsperado'] = 'NEGADO'

        empilhado = pd.concat([aprovado, recusado], axis=0)

        empilhado['Debug - Cenario'] = tipoCenario
        empilhado['Debug - idSolicitacao Massa'] = empilhado.index

        empilhado.reset_index(drop=True, inplace=True)

    return empilhado

As bibliotecas e funções utilizadas ao longo do código devem ser adicionadas aqui

In [2]:
import numpy as np
import pandas as pd
import requests
import json
import sherlock
import copy
import regras_politica as rp

###### Reload no regras_politica caso necessário

In [3]:
import importlib
import regras_politica
importlib.reload(regras_politica)

<module 'regras_politica' from 'c:\\Users\\T51657\\Desktop\\Homologação\\Modelos Online\\v2\\regras_politica.py'>

###### Base de partida

In [4]:
#Gerando a massa modelo passando o caminho do JSON como nome da variável

base_entrada = pd.DataFrame({
                            'solicitante.numeroDocumento':['12345678909'],
                            'solicitante.consumerID':[101010101],
                            'etapa':[""],
                            'solicitante.qtdIF':[1],
                            'solicitante.fonteRenda':['01. Fopag'],
                            'solicitante.statusCPF':["REGULAR"],
                            'solicitante.flagFraudePicpay':[0],
                            'solicitante.flagFraudeCartaoPicpay':[0],
                            'solicitante.flagChargebackPicpay':[0],
                            'solicitante.flagHoldForced':[0],
                            'solicitante.flagSmallLimits':[0],	
                            'solicitante.idade':[30],
                            'solicitante.atrasos.qtdDiasAtrasoCartao':[0],
                            'solicitante.atrasos.qtdDiasAtrasoParcelados':[0],
                            'solicitante.atrasos.qtdDiasAtrasoBNPLMova':[0],
                            'solicitante.atrasos.qtdDiasAtrasoP2PL':[0],
                            'solicitante.restritivosBVS.vlrMaxRestritivosBVSM1':[0],
                            'solicitante.restritivosBVS.vlrMaxRestritivosBVSM2':[0],
                            'solicitante.restritivosBVS.vlrMaxRestritivosBVSM3':[0],
                            'solicitante.vlrMaxRestritivosSerasa90Dias':[0],
                            'solicitante.flagDesenrola':[0],
                            'solicitante.flagFNV':[0],
                            'solicitante.flagReneg':[0],
                            'solicitante.flagBloqueados':[0],
                            'solicitante.rendaLiquidaPicpayBatch':[10000],
                            'solicitante.rendaLiquidaPicpay':[10000],
                            'solicitante.credilink.situacaoDocumento':['regular'],
                            'solicitante.credilink.obito':['NAO'],
                            'marcacaoMesa':["N"],

                            'flagErroAurora':[False],
                            'flagErroVisaoCliente':[False],
                            'flagErroBacen':[False],	
                            'flagErroCredilink':[False],
                            'flagErroReceitaFederal':[False],
                            'flagErroSerasa':[False],

                            'solicitante.limites.limiteDisponivel':[100000],
                            'solicitante.limites.valorContratadoCartao':[0],
                            'solicitante.maxDiasAtraso':[0],
                            'solicitante.indFNV':[False],
                            'solicitante.indReneg':[False],
                            'solicitante.flagCartaoCreditoContratado':[0],
                            'solicitante.flagLimiteGarantidoContratado':[0],
                            'solicitante.flagCartaoDebitoContratado':[0],
                            'solicitante.flagAllowlist':["-99999"],
                            'solicitante.flagBlocklist':["-99999"],
                            'solicitante.scoreAppSerasa':[999],
                            'solicitante.dataRegistro':["2002-11-05T17:18:43.888Z"],
                            'solicitante.dataAdmissaoFopag':["1001-01-01"],
                            'solicitante.dataRegistroPrimeiroContrato':["1001-01-01"],
                            'solicitante.listaRestritivosBacen.0.saldoVencido':[0],
                            'solicitante.listaRestritivosBacen.0.prejuizo':[0],
                            'solicitante.listaRestritivosBacen.0.creditoAVencerAdpos':[0],
                            'solicitante.listaRestritivosBacen.0.creditoFinanciadoCartoes':[0],
                            'solicitante.listaRestritivosBacen.0.creditoVencidoCartao':[0],
                            'solicitante.listaRestritivosBacen.0.creditoAVencerCartao':[0],
                            'solicitante.listaRestritivosBacen.0.limiteCartaoDisponivel':[0],
                            'solicitante.listaRestritivosBacen.0.limiteChequeDisponivel':[0],
                            'solicitante.listaRestritivosBacen.0.saldoBacenCheque':[0],
                            'solicitante.listaRestritivosBacen.0.renegociacaoBacen':[0],
                            'solicitante.listaRestritivosBacen.1.saldoVencido':[0],
                            'solicitante.listaRestritivosBacen.1.prejuizo':[0],
                            'solicitante.listaRestritivosBacen.1.creditoAVencerAdpos':[0],
                            'solicitante.listaRestritivosBacen.1.creditoFinanciadoCartoes':[0],
                            'solicitante.listaRestritivosBacen.1.creditoVencidoCartao':[0],
                            'solicitante.listaRestritivosBacen.1.creditoAVencerCartao':[0],
                            'solicitante.listaRestritivosBacen.1.limiteCartaoDisponivel':[0],
                            'solicitante.listaRestritivosBacen.1.limiteChequeDisponivel':[0],
                            'solicitante.listaRestritivosBacen.1.saldoBacenCheque':[0],
                            'solicitante.listaRestritivosBacen.1.renegociacaoBacen':[0],
                            'solicitante.listaRestritivosBacen.2.saldoVencido':[0],
                            'solicitante.listaRestritivosBacen.2.prejuizo':[0],
                            'solicitante.listaRestritivosBacen.2.creditoAVencerAdpos':[0],
                            'solicitante.listaRestritivosBacen.2.creditoFinanciadoCartoes':[0],
                            'solicitante.listaRestritivosBacen.2.creditoVencidoCartao':[0],
                            'solicitante.listaRestritivosBacen.2.creditoAVencerCartao':[0],
                            'solicitante.listaRestritivosBacen.2.limiteCartaoDisponivel':[0],
                            'solicitante.listaRestritivosBacen.2.limiteChequeDisponivel':[0],
                            'solicitante.listaRestritivosBacen.2.saldoBacenCheque':[0],
                            'solicitante.listaRestritivosBacen.2.renegociacaoBacen':[0],
                            'solicitante.saldoMedio':[0],
                            'solicitante.flagPrincipalidade':[1],
                            'solicitante.flagPrincipalidadeAll':[1],
                            'solicitante.mesFlagPrincipalidade':[3],
                            'solicitante.saldoMinimo30d':[101],
                            'solicitante.flagMAT30d':[0],
                            'solicitante.flagMAT35d':[0],    #adicionado nova versao
                            'solicitante.flagMAT60d':[0],
                            'solicitante.flagMAT90d':[0],
                            'solicitante.flagFuncionario':[0],
                            'solicitante.flagPortabilidade':[0],    
                            'solicitante.mediaSaldoTotal':[40000],
                            'solicitante.flagCCRot':[0],
                            'solicitante.flagInvestimentoPrincipalidade':[1],
                            'solicitante.investimentoTotal':[0],
                            'solicitante.saldoInvestimento':[0],

                            'solicitante.listaRestritivosSerasa.0.tipo':["N"],
                            'solicitante.listaRestritivosSerasa.0.montanteTotal':[0],
                            'solicitante.listaRestritivosSerasa.0.valorTotal':[0],
                            
                            
                             'solicitante.ghAppcardsBlendInternoOnline':["B"],
                             'solicitante.scoreAppcardsBlendInternoOnline':[-99999],
                             'solicitante.ghAppcardsBlendExternoOnline':["C"],
                             'solicitante.scoreAppcardsBlendExternoOnline':[-99999],
                             'solicitante.ghBlendIntExt':["E"],
                             'solicitante.scoreBlendIntExt':[-99999],
                             'solicitante.ghSerasa':["F"],
                             'solicitante.scoreSerasaAl':[-99999],
                             
                             'solicitante.blendFinalAl':[0],
                             'solicitante.ghFinalScr':["G"],
                             'solicitante.origemBlend':["04.SERASA"],

                             })

###### Criação das massas com casos forçados

In [5]:
# CENÁRIOS PARA O END TO END

# Cenários de flag tipo renda
massa_e2e_fonteRenda = sherlock.Gera_Massa(base_entrada)
massa_e2e_fonteRenda.gera_cenario('solicitante.consumerID',[90001])
massa_e2e_fonteRenda.gera_cenario('solicitante.fonteRenda',['00. Teste','01. Fopag','02. Funcionários Não FOPAG (J&F)','03. Portabilidade','04. Renda OPF Confirmada','06. Servidor Público'])
massa_e2e_fonteRenda = massa_e2e_fonteRenda.get_massa_final()

# Cenários de missing na renda
massa_e2e_missingRenda = sherlock.Gera_Massa(base_entrada)
massa_e2e_missingRenda.gera_cenario('solicitante.consumerID',[10001])
massa_e2e_missingRenda.gera_cenario('solicitante.rendaLiquidaPicpay',[10000,-99999])
massa_e2e_missingRenda.gera_cenario('solicitante.rendaLiquidaPicpayBatch',[10000,-99999])
massa_e2e_missingRenda = massa_e2e_missingRenda.get_massa_final()

# Cenário de recusa e aprovação por hard filter 
massa_e2e_hardFilterFlex = sherlock.Gera_Massa(base_entrada)
massa_e2e_hardFilterFlex.gera_cenario('solicitante.consumerID',[10002])
massa_e2e_hardFilterFlex.gera_cenario('solicitante.rendaLiquidaPicpay',[10000])
massa_e2e_hardFilterFlex.gera_cenario('solicitante.rendaLiquidaPicpayBatch',[10000])
massa_e2e_hardFilterFlex.gera_cenario('solicitante.statusCPF',["REGULAR"])
massa_e2e_hardFilterFlex.gera_cenario('solicitante.flagFraudePicpay',[0])
massa_e2e_hardFilterFlex.gera_cenario('solicitante.flagFraudeCartaoPicpay',[0])
massa_e2e_hardFilterFlex.gera_cenario('solicitante.flagHoldForced',[0])
massa_e2e_hardFilterFlex.gera_cenario('solicitante.flagChargebackPicpay',[0])
massa_e2e_hardFilterFlex.gera_cenario('solicitante.idade',[30])
massa_e2e_hardFilterFlex.gera_cenario('solicitante.atrasos.qtdDiasAtrasoCartao',[0])
massa_e2e_hardFilterFlex.gera_cenario('solicitante.atrasos.qtdDiasAtrasoParcelados',[0])
massa_e2e_hardFilterFlex.gera_cenario('solicitante.atrasos.qtdDiasAtrasoBNPLMova',[0])
massa_e2e_hardFilterFlex.gera_cenario('solicitante.atrasos.qtdDiasAtrasoP2PL',[0])
massa_e2e_hardFilterFlex.gera_cenario('solicitante.flagFNV',[0])
massa_e2e_hardFilterFlex.gera_cenario('solicitante.flagReneg',[0])
massa_e2e_hardFilterFlex.gera_cenario('solicitante.flagBloqueados',[0])
massa_e2e_hardFilterFlex.gera_cenario('solicitante.restritivosBVS.vlrMaxRestritivosBVSM1',[900,1000,2000,4000,5000,6000])  # se atentar a essa etapa uma vez que houve flexibilizacao
massa_e2e_hardFilterFlex.gera_cenario('solicitante.restritivosBVS.vlrMaxRestritivosBVSM2',[0])
massa_e2e_hardFilterFlex.gera_cenario('solicitante.restritivosBVS.vlrMaxRestritivosBVSM3',[0])
massa_e2e_hardFilterFlex.gera_cenario('solicitante.flagDesenrola',[0,1])
massa_e2e_hardFilterFlex = massa_e2e_hardFilterFlex.get_massa_final()


# Cenário de recusa e aprovação por hard filter - testa recusa pelas novas idades
massa_e2e_hardFilterFlex_2 = sherlock.Gera_Massa(base_entrada)
massa_e2e_hardFilterFlex_2.gera_cenario('solicitante.consumerID',[10002])
massa_e2e_hardFilterFlex_2.gera_cenario('solicitante.rendaLiquidaPicpay',[10000])
massa_e2e_hardFilterFlex_2.gera_cenario('solicitante.rendaLiquidaPicpayBatch',[10000])
massa_e2e_hardFilterFlex_2.gera_cenario('solicitante.idade',[17,18,79,80,81])  #testa a idade (teve alteração)
massa_e2e_hardFilterFlex_2 = massa_e2e_hardFilterFlex_2.get_massa_final()

# Cenários de recusa e aprovação na etapa de validação interna DCO
massa_validacoesInternas1 = sherlock.Gera_Massa(base_entrada)
massa_validacoesInternas1.gera_cenario('solicitante.consumerID',[1])
massa_validacoesInternas1.gera_cenario('marcacaoMesa',["N","MESA"])
massa_validacoesInternas1.gera_cenario('solicitante.limites.limiteDisponivel',[0,100000])
massa_validacoesInternas1.gera_cenario('solicitante.limites.valorContratadoCartao',[0])
massa_validacoesInternas1.gera_cenario('solicitante.maxDiasAtraso',[0])
massa_validacoesInternas1.gera_cenario('solicitante.indFNV',[False])
massa_validacoesInternas1.gera_cenario('solicitante.indReneg',[False])
massa_validacoesInternas1 = massa_validacoesInternas1.get_massa_final()

massa_validacoesInternas2 = sherlock.Gera_Massa(base_entrada)
massa_validacoesInternas2.gera_cenario('solicitante.consumerID',[2])
massa_validacoesInternas2.gera_cenario('marcacaoMesa',["N","MESA"])
massa_validacoesInternas2.gera_cenario('solicitante.limites.limiteDisponivel',[100000])
massa_validacoesInternas2.gera_cenario('solicitante.limites.valorContratadoCartao',[0,1])
massa_validacoesInternas2.gera_cenario('solicitante.maxDiasAtraso',[0])
massa_validacoesInternas2.gera_cenario('solicitante.indFNV',[False])
massa_validacoesInternas2.gera_cenario('solicitante.indReneg',[False])
massa_validacoesInternas2 = massa_validacoesInternas2.get_massa_final()

massa_validacoesInternas3 = sherlock.Gera_Massa(base_entrada)
massa_validacoesInternas3.gera_cenario('solicitante.consumerID',[3])
massa_validacoesInternas3.gera_cenario('marcacaoMesa',["N","MESA"])
massa_validacoesInternas3.gera_cenario('solicitante.limites.limiteDisponivel',[100000])
massa_validacoesInternas3.gera_cenario('solicitante.limites.valorContratadoCartao',[0])
massa_validacoesInternas3.gera_cenario('solicitante.maxDiasAtraso',[0,5,6])
massa_validacoesInternas3.gera_cenario('solicitante.indFNV',[False])
massa_validacoesInternas3.gera_cenario('solicitante.indReneg',[False])
massa_validacoesInternas3 = massa_validacoesInternas3.get_massa_final()

massa_validacoesInternas4 = sherlock.Gera_Massa(base_entrada)
massa_validacoesInternas4.gera_cenario('solicitante.consumerID',[4])
massa_validacoesInternas4.gera_cenario('marcacaoMesa',["N","MESA"])
massa_validacoesInternas4.gera_cenario('solicitante.limites.limiteDisponivel',[100000])
massa_validacoesInternas4.gera_cenario('solicitante.limites.valorContratadoCartao',[0])
massa_validacoesInternas4.gera_cenario('solicitante.maxDiasAtraso',[0,5,6])
massa_validacoesInternas4.gera_cenario('solicitante.indFNV',[False])
massa_validacoesInternas4.gera_cenario('solicitante.indReneg',[False])
massa_validacoesInternas4 = massa_validacoesInternas4.get_massa_final()

massa_validacoesInternas5 = sherlock.Gera_Massa(base_entrada)
massa_validacoesInternas5.gera_cenario('solicitante.consumerID',[5])
massa_validacoesInternas5.gera_cenario('marcacaoMesa',["N","MESA"])
massa_validacoesInternas5.gera_cenario('solicitante.limites.limiteDisponivel',[100000])
massa_validacoesInternas5.gera_cenario('solicitante.limites.valorContratadoCartao',[0])
massa_validacoesInternas5.gera_cenario('solicitante.maxDiasAtraso',[0])
massa_validacoesInternas5.gera_cenario('solicitante.indFNV',[False,True])
massa_validacoesInternas5.gera_cenario('solicitante.indReneg',[False])
massa_validacoesInternas5 = massa_validacoesInternas5.get_massa_final()

massa_validacoesInternas6 = sherlock.Gera_Massa(base_entrada)
massa_validacoesInternas6.gera_cenario('solicitante.consumerID',[6])
massa_validacoesInternas6.gera_cenario('marcacaoMesa',["N","MESA"])
massa_validacoesInternas6.gera_cenario('solicitante.limites.limiteDisponivel',[100000])
massa_validacoesInternas6.gera_cenario('solicitante.limites.valorContratadoCartao',[0])
massa_validacoesInternas6.gera_cenario('solicitante.maxDiasAtraso',[0])
massa_validacoesInternas6.gera_cenario('solicitante.indFNV',[False])
massa_validacoesInternas6.gera_cenario('solicitante.indReneg',[False,True])
massa_validacoesInternas6 = massa_validacoesInternas6.get_massa_final()

# Cenários de recusa e aprovação na etapa de filtro visão politica

#Testa Sem Info Score a partir do scoreFinal escolhido (-500,0,100)
massa_FiltroVisaoPolitica1 = sherlock.Gera_Massa(base_entrada)
massa_FiltroVisaoPolitica1.gera_cenario('solicitante.consumerID',[7])
massa_FiltroVisaoPolitica1.gera_cenario('marcacaoMesa',["N"])
massa_FiltroVisaoPolitica1.gera_cenario('solicitante.flagCartaoCreditoContratado',[0,1])
massa_FiltroVisaoPolitica1.gera_cenario('solicitante.flagLimiteGarantidoContratado',[0,1])
massa_FiltroVisaoPolitica1.gera_cenario('solicitante.flagAllowlist',["-99999"])
massa_FiltroVisaoPolitica1.gera_cenario('solicitante.flagBlocklist',["-99999"])
massa_FiltroVisaoPolitica1.gera_cenario('solicitante.blendFinalAl',[-500,0,100])   # TESTANDO SEM INFO SCORE COM 3 SCORES DIFERENTES
massa_FiltroVisaoPolitica1 = massa_FiltroVisaoPolitica1.get_massa_final()

massa_FiltroVisaoPolitica2 = sherlock.Gera_Massa(base_entrada)
massa_FiltroVisaoPolitica2.gera_cenario('solicitante.consumerID',[8])
massa_FiltroVisaoPolitica2.gera_cenario('marcacaoMesa',["N"])
massa_FiltroVisaoPolitica2.gera_cenario('solicitante.numeroDocumento',[12345678,45678912]) # TESTE RDG 6 E 7 DIGITO
massa_FiltroVisaoPolitica2.gera_cenario('solicitante.flagCartaoCreditoContratado',[0])
massa_FiltroVisaoPolitica2.gera_cenario('solicitante.flagLimiteGarantidoContratado',[0])
massa_FiltroVisaoPolitica2.gera_cenario('solicitante.flagAllowlist',["-99999","OVERALL XPTO"])
massa_FiltroVisaoPolitica2.gera_cenario('solicitante.flagBlocklist',["-99999"])
massa_FiltroVisaoPolitica2 = massa_FiltroVisaoPolitica2.get_massa_final()

massa_FiltroVisaoPolitica3 = sherlock.Gera_Massa(base_entrada)
massa_FiltroVisaoPolitica3.gera_cenario('solicitante.consumerID',[9,-99999])
massa_FiltroVisaoPolitica3.gera_cenario('marcacaoMesa',["N"])
massa_FiltroVisaoPolitica3.gera_cenario('solicitante.flagMAT35d',[1,0])    # Teste flagMat35d
massa_FiltroVisaoPolitica3.gera_cenario('solicitante.flagCartaoCreditoContratado',[0])
massa_FiltroVisaoPolitica3.gera_cenario('solicitante.flagLimiteGarantidoContratado',[0])
massa_FiltroVisaoPolitica3.gera_cenario('solicitante.flagAllowlist',["-99999","OVERALL XPTO"])
massa_FiltroVisaoPolitica3.gera_cenario('solicitante.flagBlocklist',["-99999","OVERALL XPTO"])
massa_FiltroVisaoPolitica3.gera_cenario('solicitante.dataRegistro',["2002-11-05T17:18:43.888Z","2024-04-02T17:18:43.888Z"])
massa_FiltroVisaoPolitica3 = massa_FiltroVisaoPolitica3.get_massa_final()

# # Cenários de Regras de Bacen
# massa_Bacen = sherlock.Gera_Massa(base_entrada)
# massa_Bacen.gera_cenario('solicitante.consumerID',[10])
# massa_Bacen.gera_cenario('marcacaoMesa',["N","MESA"])
# massa_Bacen.gera_cenario('solicitante.rendaLiquidaPicpay',[10000,1000])
# massa_Bacen.gera_cenario('solicitante.rendaLiquidaPicpayBatch',[10000,1000])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.0.saldoVencido',[900,1000,2000,4000,5000,6000])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.0.prejuizo',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.0.creditoAVencerAdpos',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.0.creditoFinanciadoCartoes',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.0.creditoVencidoCartao',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.0.creditoAVencerCartao',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.0.limiteCartaoDisponivel',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.0.limiteChequeDisponivel',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.0.saldoBacenCheque',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.0.renegociacaoBacen',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.1.saldoVencido',[10000])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.1.prejuizo',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.1.creditoAVencerAdpos',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.1.creditoFinanciadoCartoes',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.1.creditoVencidoCartao',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.1.creditoAVencerCartao',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.1.limiteCartaoDisponivel',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.1.limiteChequeDisponivel',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.1.saldoBacenCheque',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.1.renegociacaoBacen',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.2.saldoVencido',[10000])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.2.prejuizo',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.2.creditoAVencerAdpos',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.2.creditoFinanciadoCartoes',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.2.creditoVencidoCartao',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.2.creditoAVencerCartao',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.2.limiteCartaoDisponivel',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.2.limiteChequeDisponivel',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.2.saldoBacenCheque',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.2.renegociacaoBacen',[0])
# massa_Bacen = massa_Bacen.get_massa_final()

# # Cenários de politica concessão -  SegmentacaoNegocio = "Alta renda investidor" / SegmentacaoRisco = '1- principalidade recorrente'
# massa_politicaConcessao1 = sherlock.Gera_Massa(base_entrada)
# massa_politicaConcessao1.gera_cenario('solicitante.numeroDocumento',[11114411111,11119911111])
# massa_politicaConcessao1.gera_cenario('solicitante.consumerID',[11])
# massa_politicaConcessao1.gera_cenario('solicitante.rendaLiquidaPicpay',[3000])   #segmentacaoNegocio
# massa_politicaConcessao1.gera_cenario('solicitante.saldoMedio',[100000])         #segmentacaoNegocio
# massa_politicaConcessao1.gera_cenario('solicitante.flagPrincipalidadeAll',[1])   #segmentacaoNegocio
# massa_politicaConcessao1.gera_cenario('solicitante.flagPrincipalidade',[1])             #recorrenciaPrincipalidade        
# massa_politicaConcessao1.gera_cenario('solicitante.mesFlagPrincipalidade',[0,3])          #recorrenciaPrincipalidade     
# massa_politicaConcessao1.gera_cenario('solicitante.saldoMinimo30d',[99])          #recorrenciaPrincipalidade     
# massa_politicaConcessao1.gera_cenario('solicitante.dataRegistro',["1001-01-01","2024-04-02T17:18:43.888Z"])   #flagClienteExistente
# massa_politicaConcessao1.gera_cenario('solicitante.scoreInterno',[-99999,732,914,926,954])     
# massa_politicaConcessao1.gera_cenario('solicitante.flagFuncionario',[0,1])   
# massa_politicaConcessao1.gera_cenario('solicitante.fonteRenda',['01. Fopag','X'])  
# massa_politicaConcessao1.gera_cenario('solicitante.flagPortabilidade',[0,1])     
# massa_politicaConcessao1.gera_cenario('solicitante.flagCCRot',[0,1])         
# massa_politicaConcessao1.gera_cenario('solicitante.mediaSaldoTotal',[40000,40001])         
# massa_politicaConcessao1 = massa_politicaConcessao1.get_massa_final()

# # Cenários de Etapa Bureau
# massa_politicaBureau1 = sherlock.Gera_Massa(base_entrada)
# massa_politicaBureau1.gera_cenario('marcacaoMesa',["N","MESA"])
# massa_politicaBureau1.gera_cenario('solicitante.consumerID',[12])
# massa_politicaBureau1.gera_cenario('solicitante.credilink.situacaoDocumento',['regular','irregular','-99999'])   
# massa_politicaBureau1.gera_cenario('solicitante.credilink.obito',['N','SIM','-99999'])         
# massa_politicaBureau1 = massa_politicaBureau1.get_massa_final()


# # Cenários Serasa
# massa_politicaSerasa1 = sherlock.Gera_Massa(base_entrada)
# massa_politicaSerasa1.gera_cenario('marcacaoMesa',["N","MESA"])
# massa_politicaSerasa1.gera_cenario('solicitante.consumerID',[13])
# massa_politicaSerasa1.gera_cenario('solicitante.scoreInterno',[-99999,1,999])     
# massa_politicaSerasa1.gera_cenario('solicitante.numeroDocumento',[11111179111,11111194111,11111199111])
# massa_politicaSerasa1.gera_cenario('solicitante.rendaLiquidaPicpay',[10000,3000])
# massa_politicaSerasa1.gera_cenario('solicitante.qtdIF',[0,1,3])
# #massa_politicaSerasa1.gera_cenario('limitePoliticaConcessao',[-1]) # Não vamos forçar um valor aqui pois esse valor vira de uma etapa anterior.
# massa_politicaSerasa1.gera_cenario('solicitante.rendaLiquidaPicpay',[10000])
# massa_politicaSerasa1 = massa_politicaSerasa1.get_massa_final()

# massa_politicaSerasa2 = sherlock.Gera_Massa(base_entrada)
# massa_politicaSerasa2.gera_cenario('marcacaoMesa',["N","MESA"])
# massa_politicaSerasa2.gera_cenario('solicitante.listaRestritivosSerasa.0.tipo',['N','ACAO'])
# massa_politicaSerasa2.gera_cenario('solicitante.listaRestritivosSerasa.0.montanteTotal',[0,1,1000])     
# massa_politicaSerasa2.gera_cenario('solicitante.qtdIF',[0,1,3])
# #massa_politicaSerasa2.gera_cenario('limitePoliticaConcessao',[-1]) # Não vamos forçar um valor aqui pois esse valor vira de uma etapa anterior.
# massa_politicaSerasa2.gera_cenario('solicitante.rendaLiquidaPicpay',[10000])
# massa_politicaSerasa2 = massa_politicaSerasa2.get_massa_final()

# # Cenários de flag erro - Já existiam antes no DCO. 
# massa_politicaErros = sherlock.Gera_Massa(base_entrada)
# massa_politicaErros.gera_cenario('marcacaoMesa',["N"])
# massa_politicaErros.gera_cenario('flagErroAurora',[True,False])
# massa_politicaErros.gera_cenario('flagErroVisaoCliente',[True,False])
# massa_politicaErros.gera_cenario('flagErroBacen',[True,False])
# massa_politicaErros.gera_cenario('flagErroCredilink',[True,False])
# massa_politicaErros.gera_cenario('flagErroReceitaFederal',[True,False])
# massa_politicaErros.gera_cenario('flagErroSerasa',[True,False])
# massa_politicaErros = massa_politicaErros.get_massa_final()

# #Empilha todos casos com 1 caso padrão.
# massa_EndToEnd = sherlock.Gera_Massa(base_entrada)
# massa_EndToEnd.empilha_massas([massa_e2e_fonteRenda,
#                                massa_e2e_missingRenda,
#                                massa_e2e_hardFilterFlex,
#                                massa_validacoesInternas1,
#                                massa_validacoesInternas2,
#                                massa_validacoesInternas3,
#                                massa_validacoesInternas4,
#                                massa_validacoesInternas5,
#                                massa_validacoesInternas6,
#                                massa_FiltroVisaoPolitica1,
#                                massa_FiltroVisaoPolitica2,
#                                massa_FiltroVisaoPolitica3,
#                                massa_Bacen,
#                                massa_politicaConcessao1,
#                                massa_politicaBureau1,
#                                massa_politicaSerasa1,
#                                massa_politicaErros,
#                                massa_politicaSerasa2
# ])
                        

# massa_EndToEnd.set_bom_arqv()
# massa_EndToEnd.converte_massa()


#Empilha todos casos com 1 caso padrão.
massa_EndToEnd = sherlock.Gera_Massa(base_entrada)
massa_EndToEnd.empilha_massas([massa_e2e_fonteRenda,
                               massa_e2e_missingRenda,
                               massa_e2e_hardFilterFlex,
                               massa_e2e_hardFilterFlex_2,
                               massa_validacoesInternas1,
                               massa_validacoesInternas2,
                               massa_validacoesInternas3,
                               massa_validacoesInternas4,
                               massa_validacoesInternas5,
                               massa_validacoesInternas6,
                               massa_FiltroVisaoPolitica1,
                               massa_FiltroVisaoPolitica2,
                               massa_FiltroVisaoPolitica3
])
                        

massa_EndToEnd.set_bom_arqv()
massa_EndToEnd.converte_massa()

--------------------------------------
Quantidade de Linhas da Massa: 104
--------------------------------------
Payload setado com sucesso
--------------------------------------
Conversao concluida com sucesso


In [6]:
# # CENÁRIOS PARA O TESTE INTEGRADO ( TODOS FOPAG - VAI USAR RENDA BATCH, PORTANTO A RENDA NAO DEVE SE ALTERAR )

# # Cenários de missing na renda
# massa_e2e_missingRenda = sherlock.Gera_Massa(base_entrada)
# massa_e2e_missingRenda.gera_cenario('solicitante.consumerID',[10001])
# massa_e2e_missingRenda.gera_cenario('solicitante.rendaLiquidaPicpay',[10000])
# massa_e2e_missingRenda.gera_cenario('solicitante.rendaLiquidaPicpayBatch',[10000])
# massa_e2e_missingRenda = massa_e2e_missingRenda.get_massa_final()

# # Cenário de recusa e aprovação por hard filter
# massa_e2e_hardFilterFlex = sherlock.Gera_Massa(base_entrada)
# massa_e2e_hardFilterFlex.gera_cenario('solicitante.consumerID',[10002])
# massa_e2e_hardFilterFlex.gera_cenario('solicitante.rendaLiquidaPicpay',[10000])
# massa_e2e_hardFilterFlex.gera_cenario('solicitante.rendaLiquidaPicpayBatch',[10000])
# massa_e2e_hardFilterFlex.gera_cenario('solicitante.flagCadastroIrregular',[0])
# massa_e2e_hardFilterFlex.gera_cenario('solicitante.flagCadastroNegado',[0])
# massa_e2e_hardFilterFlex.gera_cenario('solicitante.flagCadastroEmAndamento',[0])
# massa_e2e_hardFilterFlex.gera_cenario('solicitante.flagContaInativa',[0])
# massa_e2e_hardFilterFlex.gera_cenario('solicitante.flagContaTeste',[0])
# massa_e2e_hardFilterFlex.gera_cenario('solicitante.flagOSDefasado',[0])
# massa_e2e_hardFilterFlex.gera_cenario('solicitante.statusCPF',["REGULAR"])
# massa_e2e_hardFilterFlex.gera_cenario('solicitante.flagFraudePicpay',[0])
# massa_e2e_hardFilterFlex.gera_cenario('solicitante.flagFraudeCartaoPicpay',[0])
# massa_e2e_hardFilterFlex.gera_cenario('solicitante.flagHoldForced',[0])
# massa_e2e_hardFilterFlex.gera_cenario('solicitante.flagChargebackPicpay',[0])
# massa_e2e_hardFilterFlex.gera_cenario('solicitante.idade',[30])
# massa_e2e_hardFilterFlex.gera_cenario('solicitante.atrasos.qtdDiasAtrasoCartao',[0])
# massa_e2e_hardFilterFlex.gera_cenario('solicitante.atrasos.qtdDiasAtrasoParcelados',[0])
# massa_e2e_hardFilterFlex.gera_cenario('solicitante.atrasos.qtdDiasAtrasoBNPLMova',[0])
# massa_e2e_hardFilterFlex.gera_cenario('solicitante.atrasos.qtdDiasAtrasoP2PL',[0])
# massa_e2e_hardFilterFlex.gera_cenario('solicitante.flagFNV',[0])
# massa_e2e_hardFilterFlex.gera_cenario('solicitante.flagReneg',[0])
# massa_e2e_hardFilterFlex.gera_cenario('solicitante.flagBloqueados',[0])
# massa_e2e_hardFilterFlex.gera_cenario('solicitante.restritivosBVS.vlrMaxRestritivosBVSM1',[900,1000,2000,4000,5000,6000])
# massa_e2e_hardFilterFlex.gera_cenario('solicitante.restritivosBVS.vlrMaxRestritivosBVSM2',[0])
# massa_e2e_hardFilterFlex.gera_cenario('solicitante.restritivosBVS.vlrMaxRestritivosBVSM3',[0])
# massa_e2e_hardFilterFlex.gera_cenario('solicitante.flagDesenrola',[0,1])
# massa_e2e_hardFilterFlex = massa_e2e_hardFilterFlex.get_massa_final()

# # Cenários de recusa e aprovação na etapa de validação interna DCO
# massa_validacoesInternas1 = sherlock.Gera_Massa(base_entrada)
# massa_validacoesInternas1.gera_cenario('solicitante.consumerID',[1])
# massa_validacoesInternas1.gera_cenario('marcacaoMesa',["N","MESA"])
# massa_validacoesInternas1.gera_cenario('solicitante.limites.limiteDisponivel',[0,100000])
# massa_validacoesInternas1.gera_cenario('solicitante.limites.valorContratadoCartao',[0])
# massa_validacoesInternas1.gera_cenario('solicitante.maxDiasAtraso',[0])
# massa_validacoesInternas1.gera_cenario('solicitante.indFNV',[False])
# massa_validacoesInternas1.gera_cenario('solicitante.indReneg',[False])
# massa_validacoesInternas1 = massa_validacoesInternas1.get_massa_final()

# massa_validacoesInternas2 = sherlock.Gera_Massa(base_entrada)
# massa_validacoesInternas2.gera_cenario('solicitante.consumerID',[2])
# massa_validacoesInternas2.gera_cenario('marcacaoMesa',["N","MESA"])
# massa_validacoesInternas2.gera_cenario('solicitante.limites.limiteDisponivel',[100000])
# massa_validacoesInternas2.gera_cenario('solicitante.limites.valorContratadoCartao',[0,1])
# massa_validacoesInternas2.gera_cenario('solicitante.maxDiasAtraso',[0])
# massa_validacoesInternas2.gera_cenario('solicitante.indFNV',[False])
# massa_validacoesInternas2.gera_cenario('solicitante.indReneg',[False])
# massa_validacoesInternas2 = massa_validacoesInternas2.get_massa_final()

# massa_validacoesInternas3 = sherlock.Gera_Massa(base_entrada)
# massa_validacoesInternas3.gera_cenario('solicitante.consumerID',[3])
# massa_validacoesInternas3.gera_cenario('marcacaoMesa',["N","MESA"])
# massa_validacoesInternas3.gera_cenario('solicitante.limites.limiteDisponivel',[100000])
# massa_validacoesInternas3.gera_cenario('solicitante.limites.valorContratadoCartao',[0])
# massa_validacoesInternas3.gera_cenario('solicitante.maxDiasAtraso',[0,5,6])
# massa_validacoesInternas3.gera_cenario('solicitante.indFNV',[False])
# massa_validacoesInternas3.gera_cenario('solicitante.indReneg',[False])
# massa_validacoesInternas3 = massa_validacoesInternas3.get_massa_final()

# massa_validacoesInternas4 = sherlock.Gera_Massa(base_entrada)
# massa_validacoesInternas4.gera_cenario('solicitante.consumerID',[4])
# massa_validacoesInternas4.gera_cenario('marcacaoMesa',["N","MESA"])
# massa_validacoesInternas4.gera_cenario('solicitante.limites.limiteDisponivel',[100000])
# massa_validacoesInternas4.gera_cenario('solicitante.limites.valorContratadoCartao',[0])
# massa_validacoesInternas4.gera_cenario('solicitante.maxDiasAtraso',[0,5,6])
# massa_validacoesInternas4.gera_cenario('solicitante.indFNV',[False])
# massa_validacoesInternas4.gera_cenario('solicitante.indReneg',[False])
# massa_validacoesInternas4 = massa_validacoesInternas4.get_massa_final()

# massa_validacoesInternas5 = sherlock.Gera_Massa(base_entrada)
# massa_validacoesInternas5.gera_cenario('solicitante.consumerID',[5])
# massa_validacoesInternas5.gera_cenario('marcacaoMesa',["N","MESA"])
# massa_validacoesInternas5.gera_cenario('solicitante.limites.limiteDisponivel',[100000])
# massa_validacoesInternas5.gera_cenario('solicitante.limites.valorContratadoCartao',[0])
# massa_validacoesInternas5.gera_cenario('solicitante.maxDiasAtraso',[0])
# massa_validacoesInternas5.gera_cenario('solicitante.indFNV',[False,True])
# massa_validacoesInternas5.gera_cenario('solicitante.indReneg',[False])
# massa_validacoesInternas5 = massa_validacoesInternas5.get_massa_final()

# massa_validacoesInternas6 = sherlock.Gera_Massa(base_entrada)
# massa_validacoesInternas6.gera_cenario('solicitante.consumerID',[6])
# massa_validacoesInternas6.gera_cenario('marcacaoMesa',["N","MESA"])
# massa_validacoesInternas6.gera_cenario('solicitante.limites.limiteDisponivel',[100000])
# massa_validacoesInternas6.gera_cenario('solicitante.limites.valorContratadoCartao',[0])
# massa_validacoesInternas6.gera_cenario('solicitante.maxDiasAtraso',[0])
# massa_validacoesInternas6.gera_cenario('solicitante.indFNV',[False])
# massa_validacoesInternas6.gera_cenario('solicitante.indReneg',[False,True])
# massa_validacoesInternas6 = massa_validacoesInternas6.get_massa_final()

# # Cenários de recusa e aprovação na etapa de filtro visão politica
# massa_FiltroVisaoPolitica1 = sherlock.Gera_Massa(base_entrada)
# massa_FiltroVisaoPolitica1.gera_cenario('solicitante.consumerID',[7])
# massa_FiltroVisaoPolitica1.gera_cenario('marcacaoMesa',["N"])
# massa_FiltroVisaoPolitica1.gera_cenario('solicitante.flagCartaoCreditoContratado',[0,1])
# massa_FiltroVisaoPolitica1.gera_cenario('solicitante.flagLimiteGarantidoContratado',[0,1])
# massa_FiltroVisaoPolitica1.gera_cenario('solicitante.flagAllowlist',["-99999"])
# massa_FiltroVisaoPolitica1.gera_cenario('solicitante.flagBlocklist',["-99999"])
# #massa_FiltroVisaoPolitica1.gera_cenario('solicitante.scoreAppSerasa',[999])
# massa_FiltroVisaoPolitica1.gera_cenario('solicitante.scoreInternoLongo',[999])
# massa_FiltroVisaoPolitica1.gera_cenario('solicitante.scoreInterno',[999])
# massa_FiltroVisaoPolitica1 = massa_FiltroVisaoPolitica1.get_massa_final()

# massa_FiltroVisaoPolitica2 = sherlock.Gera_Massa(base_entrada)
# massa_FiltroVisaoPolitica2.gera_cenario('solicitante.consumerID',[8])
# massa_FiltroVisaoPolitica2.gera_cenario('marcacaoMesa',["N"])
# massa_FiltroVisaoPolitica2.gera_cenario('solicitante.flagCartaoCreditoContratado',[0])
# massa_FiltroVisaoPolitica2.gera_cenario('solicitante.flagLimiteGarantidoContratado',[0])
# massa_FiltroVisaoPolitica2.gera_cenario('solicitante.flagAllowlist',["-99999"])
# massa_FiltroVisaoPolitica2.gera_cenario('solicitante.flagBlocklist',["-99999"])
# massa_FiltroVisaoPolitica2.gera_cenario('solicitante.scoreAppSerasa',[453,454,455])
# massa_FiltroVisaoPolitica2.gera_cenario('solicitante.scoreInternoLongo',[999])
# massa_FiltroVisaoPolitica2.gera_cenario('solicitante.scoreInterno',[999])
# massa_FiltroVisaoPolitica2 = massa_FiltroVisaoPolitica2.get_massa_final()

# massa_FiltroVisaoPolitica3 = sherlock.Gera_Massa(base_entrada)
# massa_FiltroVisaoPolitica3.gera_cenario('solicitante.consumerID',[9])
# massa_FiltroVisaoPolitica3.gera_cenario('marcacaoMesa',["N"])
# massa_FiltroVisaoPolitica3.gera_cenario('solicitante.flagCartaoCreditoContratado',[0])
# massa_FiltroVisaoPolitica3.gera_cenario('solicitante.flagLimiteGarantidoContratado',[0])
# massa_FiltroVisaoPolitica3.gera_cenario('solicitante.flagAllowlist',["-99999"])
# massa_FiltroVisaoPolitica3.gera_cenario('solicitante.flagBlocklist',["-99999"])
# massa_FiltroVisaoPolitica3.gera_cenario('solicitante.scoreAppSerasa',[999])
# massa_FiltroVisaoPolitica3.gera_cenario('solicitante.scoreExterno',[999])
# massa_FiltroVisaoPolitica3.gera_cenario('solicitante.scoreInterno',[999])
# massa_FiltroVisaoPolitica3.gera_cenario('solicitante.dataRegistro',["2002-11-05T17:18:43.888Z","2024-04-02T17:18:43.888Z"])
# massa_FiltroVisaoPolitica3 = massa_FiltroVisaoPolitica3.get_massa_final()

# # Cenários de Regras de Bacen
# massa_Bacen = sherlock.Gera_Massa(base_entrada)
# massa_Bacen.gera_cenario('solicitante.consumerID',[10])
# massa_Bacen.gera_cenario('marcacaoMesa',["N","MESA"])
# massa_Bacen.gera_cenario('solicitante.rendaLiquidaPicpay',[10000])
# massa_Bacen.gera_cenario('solicitante.rendaLiquidaPicpayBatch',[10000])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.0.saldoVencido',[900,1000,2000,4000,5000,6000])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.0.prejuizo',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.0.creditoAVencerAdpos',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.0.creditoFinanciadoCartoes',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.0.creditoVencidoCartao',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.0.creditoAVencerCartao',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.0.limiteCartaoDisponivel',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.0.limiteChequeDisponivel',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.0.saldoBacenCheque',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.0.renegociacaoBacen',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.1.saldoVencido',[10000])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.1.prejuizo',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.1.creditoAVencerAdpos',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.1.creditoFinanciadoCartoes',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.1.creditoVencidoCartao',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.1.creditoAVencerCartao',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.1.limiteCartaoDisponivel',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.1.limiteChequeDisponivel',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.1.saldoBacenCheque',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.1.renegociacaoBacen',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.2.saldoVencido',[10000])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.2.prejuizo',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.2.creditoAVencerAdpos',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.2.creditoFinanciadoCartoes',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.2.creditoVencidoCartao',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.2.creditoAVencerCartao',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.2.limiteCartaoDisponivel',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.2.limiteChequeDisponivel',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.2.saldoBacenCheque',[0])
# massa_Bacen.gera_cenario('solicitante.listaRestritivosBacen.2.renegociacaoBacen',[0])
# massa_Bacen = massa_Bacen.get_massa_final()

# # Cenários de politica concessão -  SegmentacaoNegocio = "Alta renda investidor" / SegmentacaoRisco = '1- principalidade recorrente'
# massa_politicaConcessao1 = sherlock.Gera_Massa(base_entrada)
# massa_politicaConcessao1.gera_cenario('solicitante.numeroDocumento',[11114411111,11119911111])
# massa_politicaConcessao1.gera_cenario('solicitante.consumerID',[11])
# massa_politicaConcessao1.gera_cenario('solicitante.rendaLiquidaPicpay',[3000])   #segmentacaoNegocio
# massa_politicaConcessao1.gera_cenario('solicitante.saldoMedio',[100000])         #segmentacaoNegocio
# massa_politicaConcessao1.gera_cenario('solicitante.flagPrincipalidadeAll',[1])   #segmentacaoNegocio
# massa_politicaConcessao1.gera_cenario('solicitante.flagPrincipalidade',[1])             #recorrenciaPrincipalidade        
# massa_politicaConcessao1.gera_cenario('solicitante.mesFlagPrincipalidade',[0,3])          #recorrenciaPrincipalidade     
# massa_politicaConcessao1.gera_cenario('solicitante.saldoMinimo30d',[99])          #recorrenciaPrincipalidade     
# massa_politicaConcessao1.gera_cenario('solicitante.dataRegistro',["1001-01-01","2024-04-02T17:18:43.888Z"])   #flagClienteExistente
# massa_politicaConcessao1.gera_cenario('solicitante.scoreInterno',[732,914,926,954])     
# massa_politicaConcessao1.gera_cenario('solicitante.flagFuncionario',[0,1])   
# massa_politicaConcessao1.gera_cenario('solicitante.fonteRenda',['01. Fopag'])  
# massa_politicaConcessao1.gera_cenario('solicitante.flagPortabilidade',[0,1])     
# massa_politicaConcessao1.gera_cenario('solicitante.flagCCRot',[0,1])         
# massa_politicaConcessao1.gera_cenario('solicitante.mediaSaldoTotal',[40000,40001])         
# massa_politicaConcessao1 = massa_politicaConcessao1.get_massa_final()

# # Cenários de Etapa Bureau
# massa_politicaBureau1 = sherlock.Gera_Massa(base_entrada)
# massa_politicaBureau1.gera_cenario('marcacaoMesa',["N","MESA"])
# massa_politicaBureau1.gera_cenario('solicitante.consumerID',[12])
# massa_politicaBureau1.gera_cenario('solicitante.credilink.situacaoDocumento',['regular','irregular'])   
# massa_politicaBureau1.gera_cenario('solicitante.credilink.obito',['N','SIM'])         
# massa_politicaBureau1 = massa_politicaBureau1.get_massa_final()


# # Cenários Serasa
# massa_politicaSerasa1 = sherlock.Gera_Massa(base_entrada)
# massa_politicaSerasa1.gera_cenario('marcacaoMesa',["N","MESA"])
# massa_politicaSerasa1.gera_cenario('solicitante.consumerID',[13])
# massa_politicaSerasa1.gera_cenario('solicitante.scoreInterno',[1,999])     
# massa_politicaSerasa1.gera_cenario('solicitante.numeroDocumento',[11111179111,11111194111,11111199111])
# massa_politicaSerasa1.gera_cenario('solicitante.qtdIF',[0,1,3])
# #massa_politicaSerasa1.gera_cenario('limitePoliticaConcessao',[-1]) # Não vamos forçar um valor aqui pois esse valor vira de uma etapa anterior.
# massa_politicaSerasa1.gera_cenario('solicitante.rendaLiquidaPicpay',[10000])
# massa_politicaSerasa1 = massa_politicaSerasa1.get_massa_final()

# massa_politicaSerasa2 = sherlock.Gera_Massa(base_entrada)
# massa_politicaSerasa2.gera_cenario('marcacaoMesa',["N","MESA"])
# massa_politicaSerasa2.gera_cenario('solicitante.listaRestritivosSerasa.0.tipo',['N','ACAO'])
# massa_politicaSerasa2.gera_cenario('solicitante.listaRestritivosSerasa.0.montanteTotal',[0,1,1000])     
# massa_politicaSerasa2.gera_cenario('solicitante.qtdIF',[0,1,3])
# #massa_politicaSerasa2.gera_cenario('limitePoliticaConcessao',[-1]) # Não vamos forçar um valor aqui pois esse valor vira de uma etapa anterior.
# massa_politicaSerasa2.gera_cenario('solicitante.rendaLiquidaPicpay',[10000])
# massa_politicaSerasa2 = massa_politicaSerasa2.get_massa_final()

# # Cenários de flag erro - Já existiam antes no DCO. 
# massa_politicaErros = sherlock.Gera_Massa(base_entrada)
# massa_politicaErros.gera_cenario('marcacaoMesa',["N"])
# massa_politicaErros.gera_cenario('flagErroAurora',[True,False])
# massa_politicaErros.gera_cenario('flagErroVisaoCliente',[True,False])
# massa_politicaErros.gera_cenario('flagErroBacen',[True,False])
# massa_politicaErros.gera_cenario('flagErroCredilink',[True,False])
# massa_politicaErros.gera_cenario('flagErroReceitaFederal',[True,False])
# massa_politicaErros.gera_cenario('flagErroSerasa',[True,False])
# massa_politicaErros = massa_politicaErros.get_massa_final()

# #Empilha todos casos com 1 caso padrão.
# massa_EndToEnd = sherlock.Gera_Massa(base_entrada)
# massa_EndToEnd.empilha_massas([
#                                massa_e2e_missingRenda,
#                                massa_e2e_hardFilterFlex,
#                                massa_validacoesInternas1,
#                                massa_validacoesInternas2,
#                                massa_validacoesInternas3,
#                                massa_validacoesInternas4,
#                                massa_validacoesInternas5,
#                                massa_validacoesInternas6,
#                                massa_FiltroVisaoPolitica1,
#                                massa_FiltroVisaoPolitica2,
#                                massa_FiltroVisaoPolitica3,
#                                massa_Bacen,
#                                massa_politicaConcessao1,
#                                massa_politicaBureau1,
#                                massa_politicaSerasa1,
#                                massa_politicaErros,
#                                massa_politicaSerasa2
# ])
                        

# massa_EndToEnd.set_bom_arqv()
# massa_EndToEnd.converte_massa()

##### Função de request no dmps

In [7]:
from multiprocessing import Pool
import requests
import json
import warnings
from urllib3.exceptions import InsecureRequestWarning
from concurrent.futures import ThreadPoolExecutor
warnings.simplefilter('ignore', InsecureRequestWarning)

class erroStatusCode(Exception):
    pass

def chamadaRestDMPS(caso, url, headers):
    payloadJson = json.dumps(caso)
    response = requests.put(url, headers=headers, data=payloadJson, verify=False)
    statusCode = response.status_code
    if statusCode != 200:
        raise erroStatusCode(f"Erro na chamada do DMPS: {response.text}")
    return response.json()

def execute_requests(casosExec, url, headers):
    num_cores = 6
    with ThreadPoolExecutor(max_workers=num_cores) as executor:
        args_list = [(caso, url, headers) for caso in casosExec]
        try:
            listaExecE2E = list(executor.map(lambda args: chamadaRestDMPS(*args), args_list))
        except erroStatusCode as e:
            print(str(e))
            return
    return listaExecE2E

###  Chamada RMA -> Hard Filter com flexibilização de percentuais ( etapa = "" )

###### Respostas esperadas

In [8]:
listaResultadoPythonHardFilter_1 = rp.execucaoModular(massa_EndToEnd.get_massa_json(),rp.etapaHardFilter)

###### Análises

In [9]:
results = []
for caso in listaResultadoPythonHardFilter_1:
    results.append(caso['payloadHomol']['saidas']['statusDecisao'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

,valor,Freq
0,APROVADO,91
1,NEGADO,13


In [10]:
print(json.dumps([caso for caso in listaResultadoPythonHardFilter_1 if caso['payloadHomol']['saidas']['statusDecisao'] == 'NEGADO'], indent=4))

[
    {
        "marcacaoMesa": "N",
        "etapa": "",
        "flagErroBacen": false,
        "flagErroSerasa": false,
        "flagErroAurora": false,
        "flagErroVisaoCliente": false,
        "flagErroReceitaFederal": false,
        "flagErroCredilink": false,
        "limitePoliticaConcessao": -1,
        "mensagemFiltro": "99. Accepted",
        "mensagemFiltroPolitica": "99. Accepted",
        "mensagemFiltroFinal": "99. Accepted",
        "solicitante": {
            "credilink": {
                "situacaoDocumento": "regular",
                "obito": "NAO"
            },
            "limites": {
                "limiteDisponivel": 100000,
                "valorContratadoCartao": 0
            },
            "atrasos": {
                "qtdDiasAtrasoCartao": 0,
                "qtdDiasAtrasoParcelados": 0,
                "qtdDiasAtrasoBNPLMova": 0,
                "qtdDiasAtrasoP2PL": 0
            },
            "restritivosBVS": {
                "vlrMaxRestritivos

###### Parâmetros da chamada no dmps

In [11]:
url = 'https://fico-dmps-lb-1479769718.us-east-1.elb.amazonaws.com/dmip-gw/dmip/request/servicosinteste12/response/servicosoutteste123'

dmpipTenantId = '1'
dmipApplicationName = 'PicPayDev'
dmipToken = '44d23b71-d44d-4759-b438-f73117e4dfbe'
contentType = 'application/vnd.fico.dmip.v3.0+text;charset=UTF-8'
cookie = 'JSESSIONID=93EAF2878176A0803A72F1BB66DA7935; AWSALB=8RWW5Bzifqu2tibNntmXzHbDN/5wGt861nvJnNq5NqSDjAlWAm1WaiGyelJP2tiqapQZeFWkU1VkATs/Qyeza8F0PJLkuNb0pelK2/xO6rgMTIeGC+Gk01II2lWX; AWSALBCORS=8RWW5Bzifqu2tibNntmXzHbDN/5wGt861nvJnNq5NqSDjAlWAm1WaiGyelJP2tiqapQZeFWkU1VkATs/Qyeza8F0PJLkuNb0pelK2/xO6rgMTIeGC+Gk01II2lWX; AWSALBTG=Snji2K/Av0g2zgKkZCj6slZu5nTpYvwBEhJn/IaPMvqZhrLbeqmMVSFj1+u17O0sILgRtRITMXapEB00gKV1hVYzDhEu0H4mnS07/vzrepSbTjKzB29JBxrV9MqiCuid/Hf7B9NE/MgY53neqXdpoLePzCB5fMAD4q6ZF5W+1q7/GIvI75s=; AWSALBTGCORS=Snji2K/Av0g2zgKkZCj6slZu5nTpYvwBEhJn/IaPMvqZhrLbeqmMVSFj1+u17O0sILgRtRITMXapEB00gKV1hVYzDhEu0H4mnS07/vzrepSbTjKzB29JBxrV9MqiCuid/Hf7B9NE/MgY53neqXdpoLePzCB5fMAD4q6ZF5W+1q7/GIvI75s='

headers = {
    'dmip-tenant-id': dmpipTenantId,
    'dmip-application-name': dmipApplicationName,
    'dmip-token': dmipToken,
    'Content-Type': contentType,
    'Cookie': cookie
}

###### Chamada no dmps

In [12]:
listaResultadoRMAHardFilter_1 = execute_requests(listaResultadoPythonHardFilter_1, url, headers)

###### Validação Resultados

In [14]:
listaValidacao = []

listaRegrasNegativas = []
# Primeira validação, verificar se tudo que foi requisitado voltou
if len(listaResultadoRMAHardFilter_1) != len(listaResultadoPythonHardFilter_1):
    print('Erro na quantidade de casos retornados')
# Caso tenha voltado a mesma quantidade de casos, verificar se os valores são iguais
else:
    # Percorremos o tamanho da lista de resultados do python, pois assim garantimos que todos nossos casos foram verificados
    for i in range(len(listaResultadoPythonHardFilter_1)):
        # Flag será True, se qualquer comparação falhar temos um problema de resultado
        validacao = True 
        validacaoComErro = []
        #print(i)
        # Verifica saídas de decisão
        if listaResultadoRMAHardFilter_1[i]['saida']['mensagemFiltro'] != listaResultadoPythonHardFilter_1[i]['payloadHomol']['saidas']['mensagemFiltro']:
            validacao = False
            validacaoComErro.append(1)
        if listaResultadoRMAHardFilter_1[i]['saida']['mensagemFiltroFinal'] != listaResultadoPythonHardFilter_1[i]['payloadHomol']['saidas']['mensagemFinal']:
            validacao = False
            validacaoComErro.append(2)
        if listaResultadoRMAHardFilter_1[i]['saida']['statusDecisao'] != listaResultadoPythonHardFilter_1[i]['payloadHomol']['saidas']['statusDecisao']:
            validacao = False
            validacaoComErro.append(3)

        # Verifica Flags
        if listaResultadoRMAHardFilter_1[i]['saida']['hasInvalidRegistration'] != listaResultadoPythonHardFilter_1[i]['payloadHomol']['saidas']['hasInvalidRegistration']:
            validacao = False
            validacaoComErro.append(4)
        if listaResultadoRMAHardFilter_1[i]['saida']['hasDeniedOngoingRegistration'] != listaResultadoPythonHardFilter_1[i]['payloadHomol']['saidas']['hasDeniedOngoingRegistration']:
            validacao = False
            validacaoComErro.append(5)
        if listaResultadoRMAHardFilter_1[i]['saida']['hasDeactivatedOrTestAccount'] != listaResultadoPythonHardFilter_1[i]['payloadHomol']['saidas']['hasDeactivatedOrTestAccount']:
            validacao = False    
            validacaoComErro.append(6)        
        if listaResultadoRMAHardFilter_1[i]['saida']['hasOutdatedOs'] != listaResultadoPythonHardFilter_1[i]['payloadHomol']['saidas']['hasOutdatedOs']:
            validacao = False   
            validacaoComErro.append(7)
        if listaResultadoRMAHardFilter_1[i]['saida']['isInvalidCpf'] != listaResultadoPythonHardFilter_1[i]['payloadHomol']['saidas']['isInvalidCpf']:
            validacao = False   
            validacaoComErro.append(8)
        if listaResultadoRMAHardFilter_1[i]['saida']['isDeadUser'] != listaResultadoPythonHardFilter_1[i]['payloadHomol']['saidas']['isDeadUser']:
            validacao = False   
            validacaoComErro.append(9)
        if listaResultadoRMAHardFilter_1[i]['saida']['isFraudPp'] != listaResultadoPythonHardFilter_1[i]['payloadHomol']['saidas']['isFraudPP']:
            validacao = False  
            validacaoComErro.append(10)
        if listaResultadoRMAHardFilter_1[i]['saida']['isChargebackPp'] != listaResultadoPythonHardFilter_1[i]['payloadHomol']['saidas']['isChargebackPP']:
            validacao = False  
            validacaoComErro.append(11)
        if listaResultadoRMAHardFilter_1[i]['saida']['isOutAgeRange'] != listaResultadoPythonHardFilter_1[i]['payloadHomol']['saidas']['isOutAgeRange']:
            validacao = False  
            validacaoComErro.append(12)
        if listaResultadoRMAHardFilter_1[i]['saida']['isDelayPp'] != listaResultadoPythonHardFilter_1[i]['payloadHomol']['saidas']['isDelayPP']:
            validacao = False  
            validacaoComErro.append(13)
        if listaResultadoRMAHardFilter_1[i]['saida']['isDelayOr'] != listaResultadoPythonHardFilter_1[i]['payloadHomol']['saidas']['isDelayOr']:
            validacao = False  
            validacaoComErro.append(14)
        if listaResultadoRMAHardFilter_1[i]['saida']['isRenegFnv'] != listaResultadoPythonHardFilter_1[i]['payloadHomol']['saidas']['isRenegFNV']:
            validacao = False  
            validacaoComErro.append(15)
        if listaResultadoRMAHardFilter_1[i]['saida']['hasRestrictionBvs'] != listaResultadoPythonHardFilter_1[i]['payloadHomol']['saidas']['hasRestrictionBVS']:
            validacao = False 
            validacaoComErro.append(16)
        if listaResultadoRMAHardFilter_1[i]['saida']['hasRestrictionSerasa'] != listaResultadoPythonHardFilter_1[i]['payloadHomol']['saidas']['hasRestrictionSerasa']:
            validacao = False 
            validacaoComErro.append(17)
        if listaResultadoRMAHardFilter_1[i]['saida']['isUserDesenrola'] != listaResultadoPythonHardFilter_1[i]['payloadHomol']['saidas']['isUserDesenrola']:
            validacao = False 
            validacaoComErro.append(18)
        if listaResultadoRMAHardFilter_1[i]['saida']['isNoHitBureaus'] != listaResultadoPythonHardFilter_1[i]['payloadHomol']['saidas']['isNoHitBureau']:
            validacao = False 
            validacaoComErro.append(19)
        
        if listaResultadoRMAHardFilter_1[i]["saida"]["statusDecisao"] == 'NEGADO' :
        
            if len(listaResultadoRMAHardFilter_1[i]["saida"]["regrasNegativas"]) == len(listaResultadoPythonHardFilter_1[i]["payloadHomol"]["saidas"]["regrasNegativas"]) :
                
                for y in range (0, len(listaResultadoPythonHardFilter_1[i]["payloadHomol"]["saidas"]["regrasNegativas"])):

                    if listaResultadoRMAHardFilter_1[i]["saida"]["regrasNegativas"][y]["nomeRegra"] != listaResultadoPythonHardFilter_1[i]["payloadHomol"]["saidas"]["regrasNegativas"][y]["nomeRegra"]:
                        listaRegrasNegativas.append("Erro no payload " + str(i) + " \nNegativa: " + str(y) + "\n Payload: " + listaResultadoPythonHardFilter_1[i]["responseBlaze"]["saida"]["regrasNegativas"][y]["nomeRegra"] + "\n Homologação: " + listaResultadoPythonHardFilter_1[i]["payloadHomol"]["saidas"]["regrasNegativas"][y]["nomeRegra"])
                        validacao = False

                    if listaResultadoRMAHardFilter_1[i]["saida"]["regrasNegativas"][y]["descRegra"] != listaResultadoPythonHardFilter_1[i]["payloadHomol"]["saidas"]["regrasNegativas"][y]["descricao"]:
                        listaRegrasNegativas.append("Erro no payload " + str(i) + " \nNegativa: " + str(y) + "\n Payload: " + listaResultadoPythonHardFilter_1[i]["responseBlaze"]["saida"]["regrasNegativas"][y]["descRegra"] + "\n Homologação: " + listaResultadoPythonHardFilter_1[i]["payloadHomol"]["saidas"]["regrasNegativas"][y]["descricao"])
                        flagSucesso = False
            else :
                listaRegrasNegativas.append("Divergência na quantidade de regrasNegativas do payload: " + str(i) + "\nPayload:\n " + str(listaResultadoRMAHardFilter_1[i]["saida"]["regrasNegativas"]) + "\nHomologação: \n" + str(listaResultadoPythonHardFilter_1[i]["payloadHomol"]["saidas"]["regrasNegativas"]))
                
            
        
        
        
        
        # Verifica se a próxima etapa está sendo aplicada
        if listaResultadoRMAHardFilter_1[i]['saida']['statusDecisao'] == "NEGADO":
            if listaResultadoRMAHardFilter_1[i]['saida']['etapa'] != "FIM":
                validacao = False
                validacaoComErro.append(20)
        if listaResultadoRMAHardFilter_1[i]['saida']['statusDecisao'] == "APROVADO":
            if listaResultadoRMAHardFilter_1[i]['saida']['etapa'] != "FILTROS POLITICA":
                validacao = False
                validacaoComErro.append(21)

        objetoValidacao = {
            'caso': i,
            'validacao': validacao,
            'debugValidacao': validacaoComErro,
            'request': listaResultadoPythonHardFilter_1[i],
            'response': listaResultadoRMAHardFilter_1[i]
        }

        listaValidacao.append(objetoValidacao)

countOK = 0
countNOK = 0
for validacao in listaValidacao:
    if validacao['validacao'] == True:
        countOK = countOK + 1
    else:
        countNOK = countNOK + 1
        if countNOK <= 10:
            print(f'Caso {validacao["caso"]} com erro')

print('Etapa Hard Filter - Flexibilizado')
print(f'Quantidade de cenários Totais:' ,len(listaValidacao))
print(f'Cenários Validados: {countOK} ')
print(f'Cenários com Divergência: {countNOK}')
print(f'Regras Negativas com divergencia:', len(listaRegrasNegativas))

Etapa Hard Filter - Flexibilizado
Quantidade de cenários Totais: 104
Cenários Validados: 104 
Cenários com Divergência: 0
Regras Negativas com divergencia 0


#### Se existir divergencias, procura pelo indice printado acima

In [ ]:
## buscando divergencia 
print(json.dumps(listaResultadoRMAHardFilter_1[23], indent=4))

{
    "etapa": "",
    "solicitante": {
        "restritivosBVS": {
            "vlrMaxRestritivosBVSM1": 0.0,
            "vlrMaxRestritivosBVSM2": 0.0,
            "vlrMaxRestritivosBVSM3": 0.0
        },
        "atrasos": {
            "qtdDiasAtrasoCartao": 0,
            "qtdDiasAtrasoParcelados": 0,
            "qtdDiasAtrasoBNPLMova": 0,
            "qtdDiasAtrasoP2PL": 0
        },
        "flagCadastroIrregular": 0,
        "flagCadastroNegado": 0,
        "flagCadastroEmAndamento": 0,
        "flagContaInativa": 0,
        "flagContaTeste": 0,
        "flagOSDefasado": 0,
        "flagFraudeCartaoPicpay": 0,
        "flagFraudePicpay": 0,
        "flagChargebackPicpay": 0,
        "flagBloqueados": 0,
        "idade": 17,
        "flagFNV": 0,
        "flagReneg": 0,
        "flagDesenrola": 0,
        "vlrMaxRestritivosSerasa90Dias": 0.0,
        "rendaLiquidaPicpay": 10000.0,
        "rendaLiquidaPicpayBatch": 10000.0,
        "statusCPF": "REGULAR",
        "flagHoldForce

In [ ]:
print(json.dumps(listaResultadoPythonHardFilter_1[23], indent=4))

{
    "marcacaoMesa": "N",
    "etapa": "",
    "flagErroBacen": false,
    "flagErroSerasa": false,
    "flagErroAurora": false,
    "flagErroVisaoCliente": false,
    "flagErroReceitaFederal": false,
    "flagErroCredilink": false,
    "limitePoliticaConcessao": -1,
    "mensagemFiltro": "99. Accepted",
    "mensagemFiltroPolitica": "99. Accepted",
    "mensagemFiltroFinal": "99. Accepted",
    "solicitante": {
        "credilink": {
            "situacaoDocumento": "regular",
            "obito": "NAO"
        },
        "limites": {
            "limiteDisponivel": 100000,
            "valorContratadoCartao": 0
        },
        "atrasos": {
            "qtdDiasAtrasoCartao": 0,
            "qtdDiasAtrasoParcelados": 0,
            "qtdDiasAtrasoBNPLMova": 0,
            "qtdDiasAtrasoP2PL": 0
        },
        "restritivosBVS": {
            "vlrMaxRestritivosBVSM1": 0,
            "vlrMaxRestritivosBVSM2": 0,
            "vlrMaxRestritivosBVSM3": 0
        },
        "ghAppcards

##### Procurando por itens especificos

In [ ]:
print(json.dumps([caso for caso in listaResultadoRMAHardFilter_1 if caso['solicitante']['idade'] == 79], indent=4))

[
    {
        "etapa": "",
        "solicitante": {
            "restritivosBVS": {
                "vlrMaxRestritivosBVSM1": 0.0,
                "vlrMaxRestritivosBVSM2": 0.0,
                "vlrMaxRestritivosBVSM3": 0.0
            },
            "atrasos": {
                "qtdDiasAtrasoCartao": 0,
                "qtdDiasAtrasoParcelados": 0,
                "qtdDiasAtrasoBNPLMova": 0,
                "qtdDiasAtrasoP2PL": 0
            },
            "flagCadastroIrregular": 0,
            "flagCadastroNegado": 0,
            "flagCadastroEmAndamento": 0,
            "flagContaInativa": 0,
            "flagContaTeste": 0,
            "flagOSDefasado": 0,
            "flagFraudeCartaoPicpay": 0,
            "flagFraudePicpay": 0,
            "flagChargebackPicpay": 0,
            "flagBloqueados": 0,
            "idade": 79,
            "flagFNV": 0,
            "flagReneg": 0,
            "flagDesenrola": 0,
            "vlrMaxRestritivosSerasa90Dias": 0.0,
            "rendaL

###### Extrai um caso Aprovado e um Negado dessa etapa

In [ ]:
# Parametros: Objeto de massa, lista de resultados da etapa, nome da etapa ( string para sinalizar na massa ), retorno é um dataframe
cenariosHardFilter_Etapa1 = arquivoExemplos(massa_EndToEnd.get_massa_final(),listaResultadoPythonHardFilter_1,'Hard Filter Flexibilizado')
cenariosHardFilter_Etapa1

,solicitante.numeroDocumento,solicitante.consumerID,etapa,solicitante.qtdIF,solicitante.fonteRenda,solicitante.statusCPF,solicitante.flagFraudePicpay,solicitante.flagFraudeCartaoPicpay,solicitante.flagChargebackPicpay,solicitante.flagHoldForced,...,solicitante.ghBlendIntExt,solicitante.scoreBlendIntExt,solicitante.ghSerasa,solicitante.scoreSerasaAl,solicitante.blendFinalAl,solicitante.ghFinalScr,solicitante.origemBlend,Debug - ResultadoEsperado,Debug - Cenario,Debug - idSolicitacao Massa
0,12345678909,101010101,,1,01. Fopag,REGULAR,0,0,0,0,...,E,-99999,F,-99999,0,G,04.SERASA,APROVADO,Hard Filter Flexibilizado,0
1,12345678909,10001,,1,01. Fopag,REGULAR,0,0,0,0,...,E,-99999,F,-99999,0,G,04.SERASA,NEGADO,Hard Filter Flexibilizado,8


### Chamada RMA -> FILTROS POLITICA ( Validações Internas DCO + Filtros Visão Politica )  etapa = "FILTROS POLITICA" 

###### Remove os cenários recusados na etapa anterior, seta a etapa politica para a próxima execução e também transfere as saídas do RMA para a entrada para próxima execução

In [ ]:
listaCenariosFiltrosPolitica = copy.deepcopy(listaResultadoPythonHardFilter_1)
listaCenariosFiltrosPoliticaAprovados = []
for i in range(len(listaCenariosFiltrosPolitica)):
    if listaCenariosFiltrosPolitica[i]['payloadHomol']['saidas']['statusDecisao'] == "APROVADO":
        listaCenariosFiltrosPolitica[i]['etapa'] = "FILTROS POLITICA"
        listaCenariosFiltrosPolitica[i]['etapa'] = "FILTROS POLITICA"
        listaCenariosFiltrosPoliticaAprovados.append(listaCenariosFiltrosPolitica[i])

print(f'Quantidade de cenários totais após o filtro:', len(listaCenariosFiltrosPoliticaAprovados))

Quantidade de cenários totais após o filtro: 91


###### Respostas esperadas

In [ ]:
listaResultadoPythonFiltrosPolitica = rp.execucaoModular(listaCenariosFiltrosPoliticaAprovados,rp.etapaFiltrosPolitica)

In [ ]:
for caso in listaResultadoPythonFiltrosPolitica:
    if caso['idSolicitacao'] == 14:
        print(json.dumps(caso,indent=4))

###### Análises

In [ ]:
results = []
for caso in listaResultadoPythonFiltrosPolitica:
    results.append(caso['payloadHomol']['saidas']['statusDecisao'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

,valor,Freq
0,APROVADO,67
1,NEGADO,24


In [ ]:
results = []
for caso in listaResultadoPythonFiltrosPolitica:
    results.append(caso['payloadHomol']['saidas']['flagTipoRenda'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

,valor,Freq
0,BATCH,88
1,OPEN FINANCE MODELO,1
2,OPEN FINANCE,1
3,OPEN FINANCE SERVIDOR,1


###### Parâmetros da chamada no dmps

In [ ]:
url = 'https://fico-dmps-lb-1479769718.us-east-1.elb.amazonaws.com/dmip-gw/dmip/request/servicosinteste13/response/servicosoutteste133'

dmpipTenantId = '1'
dmipApplicationName = 'PicPayDev'
dmipToken = '44d23b71-d44d-4759-b438-f73117e4dfbe'
contentType = 'application/vnd.fico.dmip.v3.0+text;charset=UTF-8'
cookie = 'JSESSIONID=93EAF2878176A0803A72F1BB66DA7935; AWSALB=8RWW5Bzifqu2tibNntmXzHbDN/5wGt861nvJnNq5NqSDjAlWAm1WaiGyelJP2tiqapQZeFWkU1VkATs/Qyeza8F0PJLkuNb0pelK2/xO6rgMTIeGC+Gk01II2lWX; AWSALBCORS=8RWW5Bzifqu2tibNntmXzHbDN/5wGt861nvJnNq5NqSDjAlWAm1WaiGyelJP2tiqapQZeFWkU1VkATs/Qyeza8F0PJLkuNb0pelK2/xO6rgMTIeGC+Gk01II2lWX; AWSALBTG=Snji2K/Av0g2zgKkZCj6slZu5nTpYvwBEhJn/IaPMvqZhrLbeqmMVSFj1+u17O0sILgRtRITMXapEB00gKV1hVYzDhEu0H4mnS07/vzrepSbTjKzB29JBxrV9MqiCuid/Hf7B9NE/MgY53neqXdpoLePzCB5fMAD4q6ZF5W+1q7/GIvI75s=; AWSALBTGCORS=Snji2K/Av0g2zgKkZCj6slZu5nTpYvwBEhJn/IaPMvqZhrLbeqmMVSFj1+u17O0sILgRtRITMXapEB00gKV1hVYzDhEu0H4mnS07/vzrepSbTjKzB29JBxrV9MqiCuid/Hf7B9NE/MgY53neqXdpoLePzCB5fMAD4q6ZF5W+1q7/GIvI75s='

headers = {
    'dmip-tenant-id': dmpipTenantId,
    'dmip-application-name': dmipApplicationName,
    'dmip-token': dmipToken,
    'Content-Type': contentType,
    'Cookie': cookie
}

###### Chamada no dmps

In [ ]:
listaResultadoRMAFiltrosPolitica = execute_requests(listaResultadoPythonFiltrosPolitica, url, headers)

###### Validação Resultados

In [ ]:
listaValidacao = []
# Primeira validação, verificar se tudo que foi requisitado voltou
if len(listaResultadoRMAFiltrosPolitica) != len(listaResultadoPythonFiltrosPolitica):
    print('Erro na quantidade de casos retornados')
# Caso tenha voltado a mesma quantidade de casos, verificar se os valores são iguais
else:
    # Percorremos o tamanho da lista de resultados do python, pois assim garantimos que todos nossos casos foram verificados
    for i in range(len(listaResultadoPythonFiltrosPolitica)):
        #print(i)
        # Flag será True, se qualquer comparação falhar temos um problema de resultado
        validacao = True 
        validacaoComErro = []

        # Verifica flagTipoRenda
        if listaResultadoRMAFiltrosPolitica[i]['saida']['flagTipoRenda'] != listaResultadoPythonFiltrosPolitica[i]['payloadHomol']['saidas']['flagTipoRenda']:
            validacao = False
            validacaoComErro.append(1)

        # Verifica statusDecisao
        if listaResultadoRMAFiltrosPolitica[i]['saida']['statusDecisao'] != listaResultadoPythonFiltrosPolitica[i]['payloadHomol']['saidas']['statusDecisao']:
            validacao = False
            validacaoComErro.append(2)

        # Faremos a verificação de motivos apenas se o statusDecisão estiver OK, caso contrário já existe a divergência por statusDecisao.
        else:
            # apenas casos negados vai existir o vetor de regras negativas, portanto fazemos a verificação apenas para esses casos.
            if (listaResultadoRMAFiltrosPolitica[i]['saida']['statusDecisao'] == "NEGADO" or listaResultadoPythonFiltrosPolitica[i]['payloadHomol']['saidas']['statusDecisao'] == "NEGADO"):
                # Verifica se existe lista de negativas nos dois lados
                if (('regrasNegativas' in listaResultadoRMAFiltrosPolitica[i]['saida'] and 
                    len(listaResultadoPythonFiltrosPolitica[i]['payloadHomol']['saidas']['regrasNegativas']) == 0
                    ) 
                    or
                    (
                    'regrasNegativas' not in listaResultadoRMAFiltrosPolitica[i]['saida'] and 
                     len(listaResultadoPythonFiltrosPolitica[i]['payloadHomol']['saidas']['regrasNegativas']) != 0
                    )
                   ):
                    validacao = False
                    validacaoComErro.append(3)
                else:
                    # Se existir a lista de negativas nos dois lados, verificamos se os motivos são iguais
                    if ('regrasNegativas' in listaResultadoRMAFiltrosPolitica[i]['saida'] and len(listaResultadoPythonFiltrosPolitica[i]['payloadHomol']['saidas']['regrasNegativas']) > 0):
                        # Verifica se a quantidade de motivos é igual
                        if len(listaResultadoRMAFiltrosPolitica[i]['saida']['regrasNegativas']) != len(listaResultadoPythonFiltrosPolitica[i]['payloadHomol']['saidas']['regrasNegativas']):
                            validacao = False
                            validacaoComErro.append(4)
                        # varre cada motivo dos dois lados e compara
                        else:
                            for j in range(len(listaResultadoRMAFiltrosPolitica[i]['saida']['regrasNegativas'])):
                                if listaResultadoRMAFiltrosPolitica[i]['saida']['regrasNegativas'][j]['descRegra'] != listaResultadoPythonFiltrosPolitica[i]['payloadHomol']['saidas']['regrasNegativas'][j]['descricao']:
                                    validacao = False
                                    validacaoComErro.append(4)
                                if listaResultadoRMAFiltrosPolitica[i]['saida']['regrasNegativas'][j]['nomeRegra'] != listaResultadoPythonFiltrosPolitica[i]['payloadHomol']['saidas']['regrasNegativas'][j]['nomeRegra']:
                                    validacao = False
                                    validacaoComErro.append(5)
                            # Verifica mensagem filtro politica ( só vai ocorrer se não for recusado na etapa de validações internas)
                            if len(listaResultadoRMAFiltrosPolitica[i]['saida']['regrasNegativas']) == 0:
                                if listaResultadoRMAFiltrosPolitica[i]['saida']['mensagemFiltroPolitica'] != listaResultadoPythonFiltrosPolitica[i]['payloadHomol']['saidas']['mensagemFinal']:
                                    validacao = False
                                    validacaoComErro.append(6)
            
        # Verifica se a próxima etapa está sendo aplicada
        if listaResultadoRMAFiltrosPolitica[i]['saida']['statusDecisao'] == "NEGADO":
            if listaResultadoRMAFiltrosPolitica[i]['saida']['etapa'] != "FIM":
                validacao = False
                validacaoComErro.append(7)
        if listaResultadoRMAFiltrosPolitica[i]['saida']['statusDecisao'] == "APROVADO":
            if listaResultadoRMAFiltrosPolitica[i]['saida']['etapa'] != "BACEN M-1":
                validacao = False
                validacaoComErro.append(8)

        objetoValidacao = {
            'caso': i,
            'validacao': validacao,
            'debugValidacao': validacaoComErro,
            'request': listaResultadoPythonFiltrosPolitica[i],
            'response': listaResultadoRMAFiltrosPolitica[i]
        }

        listaValidacao.append(objetoValidacao)

countOK = 0
countNOK = 0
for validacao in listaValidacao:
    if validacao['validacao'] == True:
        countOK = countOK + 1
    else:
        countNOK = countNOK + 1
        if countNOK <= 5:
            print(f'Caso {validacao["caso"]} com erro')
            print(f'flagsDebug: {validacao["debugValidacao"]}')
            print('------------------------------------------')

print('Etapa Validaçoes Internas DCO + Filtros Visão Politica')
print(f'Quantidade de cenários Totais:' ,len(listaValidacao))
print(f'Cenários Validados: {countOK} ')
print(f'Cenários com Divergência: {countNOK}')

Caso 1 com erro
flagsDebug: [1]
------------------------------------------
Caso 43 com erro
flagsDebug: [2]
------------------------------------------
Etapa Validaçoes Internas DCO + Filtros Visão Politica
Quantidade de cenários Totais: 91
Cenários Validados: 89 
Cenários com Divergência: 2


###### Extrai um caso Aprovado e um Negado dessa etapa

In [ ]:
# Parametros: Objeto de massa, lista de resultados da etapa, nome da etapa ( string para sinalizar na massa ), retorno é um dataframe
cenariosFiltrosPolitica = arquivoExemplos(massa_EndToEnd.get_massa_final(),listaResultadoPythonFiltrosPolitica,'Filtros Politica')
cenariosFiltrosPolitica

,solicitante.numeroDocumento,solicitante.consumerID,etapa,solicitante.qtdIF,solicitante.fonteRenda,solicitante.statusCPF,solicitante.flagFraudePicpay,solicitante.flagFraudeCartaoPicpay,solicitante.flagChargebackPicpay,solicitante.flagHoldForced,...,solicitante.ghBlendIntExt,solicitante.scoreBlendIntExt,solicitante.ghSerasa,solicitante.scoreSerasaAl,solicitante.blendFinalAl,solicitante.ghFinalScr,solicitante.origemBlend,Debug - ResultadoEsperado,Debug - Cenario,Debug - idSolicitacao Massa
0,12345678909,101010101,,1,01. Fopag,REGULAR,0,0,0,0,...,E,-99999,F,-99999,0,G,04.SERASA,APROVADO,Filtros Politica,0
1,12345678909,1,,1,01. Fopag,REGULAR,0,0,0,0,...,E,-99999,F,-99999,0,G,04.SERASA,NEGADO,Filtros Politica,28


### Chamada RMA -> BACEN DCO ( BACEN FLEXIBILIZADO )  etapa = "BACEN M-1" 

###### Remove os cenários recusados na etapa anterior, seta a etapa politica para a próxima execução e também transfere as saídas do RMA para a entrada para próxima execução

In [ ]:
listaCenariosBacen = copy.deepcopy(listaResultadoPythonFiltrosPolitica)
listaCenariosBacenAprovados = []
for i in range(len(listaCenariosBacen)):
    if listaCenariosBacen[i]['payloadHomol']['saidas']['statusDecisao'] == "APROVADO":
        listaCenariosBacen[i]['etapa'] = "BACEN M-1"
        listaCenariosBacenAprovados.append(listaCenariosBacen[i])

print(f'Quantidade de cenários totais após o filtro:', len(listaCenariosBacenAprovados))

Quantidade de cenários totais após o filtro: 67


###### Respostas esperadas

In [ ]:
listaResultadoBacenPolitica = rp.execucaoModular(listaCenariosBacenAprovados,rp.etapaBacenFlex)

In [ ]:
results = []
for caso in listaResultadoBacenPolitica:
    results.append(caso['marcacaoMesa'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

,valor,Freq
0,N,53
1,MESA,14


In [ ]:
results = []
for caso in listaResultadoBacenPolitica:
    results.append(caso['payloadHomol']['saidas']['statusDecisao'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

,valor,Freq
0,APROVADO,67


###### Parâmetros da chamada no dmps

In [ ]:
url = 'https://fico-dmps-lb-1479769718.us-east-1.elb.amazonaws.com/dmip-gw/dmip/request/servicosinteste13/response/servicosoutteste133'

dmpipTenantId = '1'
dmipApplicationName = 'PicPayDev'
dmipToken = '44d23b71-d44d-4759-b438-f73117e4dfbe'
contentType = 'application/vnd.fico.dmip.v3.0+text;charset=UTF-8'
cookie = 'JSESSIONID=93EAF2878176A0803A72F1BB66DA7935; AWSALB=8RWW5Bzifqu2tibNntmXzHbDN/5wGt861nvJnNq5NqSDjAlWAm1WaiGyelJP2tiqapQZeFWkU1VkATs/Qyeza8F0PJLkuNb0pelK2/xO6rgMTIeGC+Gk01II2lWX; AWSALBCORS=8RWW5Bzifqu2tibNntmXzHbDN/5wGt861nvJnNq5NqSDjAlWAm1WaiGyelJP2tiqapQZeFWkU1VkATs/Qyeza8F0PJLkuNb0pelK2/xO6rgMTIeGC+Gk01II2lWX; AWSALBTG=Snji2K/Av0g2zgKkZCj6slZu5nTpYvwBEhJn/IaPMvqZhrLbeqmMVSFj1+u17O0sILgRtRITMXapEB00gKV1hVYzDhEu0H4mnS07/vzrepSbTjKzB29JBxrV9MqiCuid/Hf7B9NE/MgY53neqXdpoLePzCB5fMAD4q6ZF5W+1q7/GIvI75s=; AWSALBTGCORS=Snji2K/Av0g2zgKkZCj6slZu5nTpYvwBEhJn/IaPMvqZhrLbeqmMVSFj1+u17O0sILgRtRITMXapEB00gKV1hVYzDhEu0H4mnS07/vzrepSbTjKzB29JBxrV9MqiCuid/Hf7B9NE/MgY53neqXdpoLePzCB5fMAD4q6ZF5W+1q7/GIvI75s='

headers = {
    'dmip-tenant-id': dmpipTenantId,
    'dmip-application-name': dmipApplicationName,
    'dmip-token': dmipToken,
    'Content-Type': contentType,
    'Cookie': cookie
}

###### Chamada no dmps

In [ ]:
listaResultadoRMABacen = execute_requests(listaResultadoBacenPolitica, url, headers)

###### Validação Resultados

In [ ]:
listaValidacao = []
# Primeira validação, verificar se tudo que foi requisitado voltou
if len(listaResultadoRMABacen) != len(listaResultadoBacenPolitica):
    print('Erro na quantidade de casos retornados')
# Caso tenha voltado a mesma quantidade de casos, verificar se os valores são iguais
else:
    # Percorremos o tamanho da lista de resultados do python, pois assim garantimos que todos nossos casos foram verificados
    for i in range(len(listaResultadoBacenPolitica)):
        #print(i)
        # Flag será True, se qualquer comparação falhar temos um problema de resultado
        validacao = True 
        validacaoComErro = []

        # Verifica statusDecisao
        if listaResultadoRMABacen[i]['saida']['statusDecisao'] != listaResultadoBacenPolitica[i]['payloadHomol']['saidas']['statusDecisao']:
            validacao = False
            validacaoComErro.append(1)

        # Faremos a verificação de motivos apenas se o statusDecisão estiver OK, caso contrário já existe a divergência por statusDecisao.
        else:
            # apenas casos negados vai existir o vetor de regras negativas, portanto fazemos a verificação apenas para esses casos.
            if (listaResultadoRMABacen[i]['saida']['statusDecisao'] == "NEGADO" or listaResultadoBacenPolitica[i]['payloadHomol']['saidas']['statusDecisao'] == "NEGADO"):
                # Verifica se existe lista de negativas nos dois lados
                if (('regrasNegativas' in listaResultadoRMABacen[i]['saida'] and 
                    len(listaResultadoBacenPolitica[i]['payloadHomol']['saidas']['regrasNegativas']) == 0
                    ) 
                    or
                    (
                    'regrasNegativas' not in listaResultadoRMABacen[i]['saida'] and 
                     len(listaResultadoBacenPolitica[i]['payloadHomol']['saidas']['regrasNegativas']) != 0
                    )
                   ):
                    validacao = False
                    validacaoComErro.append(2)
                else:
                    # Se existir a lista de negativas nos dois lados, verificamos se os motivos são iguais
                    if ('regrasNegativas' in listaResultadoRMABacen[i]['saida'] and len(listaResultadoBacenPolitica[i]['payloadHomol']['saidas']['regrasNegativas']) > 0):
                        # Verifica se a quantidade de motivos é igual
                        if len(listaResultadoRMABacen[i]['saida']['regrasNegativas']) != len(listaResultadoBacenPolitica[i]['payloadHomol']['saidas']['regrasNegativas']):
                            validacao = False
                            validacaoComErro.append(3)
                        # Se tamanho igual, varre cada motivo dos dois lados e compara
                        else:
                            for j in range(len(listaResultadoRMABacen[i]['saida']['regrasNegativas'])):
                                if listaResultadoRMABacen[i]['saida']['regrasNegativas'][j]['descRegra'] != listaResultadoBacenPolitica[i]['payloadHomol']['saidas']['regrasNegativas'][j]['descricao']:
                                    validacao = False
                                    validacaoComErro.append(4)
                                if listaResultadoRMABacen[i]['saida']['regrasNegativas'][j]['nomeRegra'] != listaResultadoBacenPolitica[i]['payloadHomol']['saidas']['regrasNegativas'][j]['nomeRegra']:
                                    validacao = False
                                    validacaoComErro.append(5)
            
        # Verifica se a próxima etapa está sendo aplicada
        if listaResultadoRMABacen[i]['saida']['statusDecisao'] == "NEGADO":
            if listaResultadoRMABacen[i]['saida']['etapa'] != "FIM":
                validacao = False
                validacaoComErro.append(6)
        if listaResultadoRMABacen[i]['saida']['statusDecisao'] == "APROVADO":
            if listaResultadoRMABacen[i]['saida']['etapa'] != "BACEN M-2, M-3":
                validacao = False
                validacaoComErro.append(7)

        objetoValidacao = {
            'caso': i,
            'validacao': validacao,
            'debugValidacao': validacaoComErro,
            'request': listaResultadoPythonFiltrosPolitica[i],
            'response': listaResultadoRMAFiltrosPolitica[i]
        }

        listaValidacao.append(objetoValidacao)

countOK = 0
countNOK = 0
for validacao in listaValidacao:
    if validacao['validacao'] == True:
        countOK = countOK + 1
    else:
        countNOK = countNOK + 1
        if countNOK <= 5:
            print(f'Caso {validacao["caso"]} com erro')
            print(f'flagsDebug: {validacao["debugValidacao"]}')
            print('------------------------------------------')

print('Etapa BACEN - Flexibilizado')
print(f'Quantidade de cenários Totais:' ,len(listaValidacao))
print(f'Cenários Validados: {countOK} ')
print(f'Cenários com Divergência: {countNOK}')

Etapa BACEN - Flexibilizado
Quantidade de cenários Totais: 67
Cenários Validados: 67 
Cenários com Divergência: 0


###### Extrai um caso Aprovado e um Negado dessa etapa

In [ ]:
# Parametros: Objeto de massa, lista de resultados da etapa, nome da etapa ( string para sinalizar na massa ), retorno é um dataframe
cenariosBacenFlexibilizado = arquivoExemplos(massa_EndToEnd.get_massa_final(),listaResultadoBacenPolitica,'Bacen DCO Flexibilizado')
cenariosBacenFlexibilizado

Não foi possível encontrar um cenário aprovado e um cenário negado


UnboundLocalError: cannot access local variable 'empilhado' where it is not associated with a value

### Chamada RMA -> HARD FILTER  ( NÃO FLEXIBILIZADO )  etapa = "BACEN M-2, M-3" 

###### Remove os cenários recusados na etapa anterior, seta a etapa politica para a próxima execução e também transfere as saídas do RMA para a entrada para próxima execução

In [ ]:
listaCenariosHardFilterSemFlex = copy.deepcopy(listaResultadoBacenPolitica)
listaCenariosHardFilterSemFlexAprovados = []
for i in range(len(listaCenariosHardFilterSemFlex)):
    if listaCenariosHardFilterSemFlex[i]['payloadHomol']['saidas']['statusDecisao'] == "APROVADO":
        listaCenariosHardFilterSemFlex[i]['etapa'] = "BACEN M-2, M-3"
        listaCenariosHardFilterSemFlexAprovados.append(listaCenariosHardFilterSemFlex[i])

print(f'Quantidade de cenários totais após o filtro:', len(listaCenariosHardFilterSemFlexAprovados))

Quantidade de cenários totais após o filtro: 642


###### Respostas esperadas

In [ ]:
listaResultadoPythonHardFilterSemFlex = rp.execucaoModular(listaCenariosHardFilterSemFlexAprovados,rp.etapaHardFilter)

In [ ]:
results = []
for caso in listaResultadoPythonHardFilterSemFlex:
    results.append(caso['payloadHomol']['saidas']['statusDecisao'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

,valor,Freq
0,APROVADO,639
1,NEGADO,3


###### Parâmetros da chamada no dmps

In [ ]:
url = 'https://fico-dmps-lb-1479769718.us-east-1.elb.amazonaws.com/dmip-gw/dmip/request/servicosinteste12/response/servicosoutteste123'

dmpipTenantId = '1'
dmipApplicationName = 'PicPayDev'
dmipToken = '44d23b71-d44d-4759-b438-f73117e4dfbe'
contentType = 'application/vnd.fico.dmip.v3.0+text;charset=UTF-8'
cookie = 'JSESSIONID=93EAF2878176A0803A72F1BB66DA7935; AWSALB=8RWW5Bzifqu2tibNntmXzHbDN/5wGt861nvJnNq5NqSDjAlWAm1WaiGyelJP2tiqapQZeFWkU1VkATs/Qyeza8F0PJLkuNb0pelK2/xO6rgMTIeGC+Gk01II2lWX; AWSALBCORS=8RWW5Bzifqu2tibNntmXzHbDN/5wGt861nvJnNq5NqSDjAlWAm1WaiGyelJP2tiqapQZeFWkU1VkATs/Qyeza8F0PJLkuNb0pelK2/xO6rgMTIeGC+Gk01II2lWX; AWSALBTG=Snji2K/Av0g2zgKkZCj6slZu5nTpYvwBEhJn/IaPMvqZhrLbeqmMVSFj1+u17O0sILgRtRITMXapEB00gKV1hVYzDhEu0H4mnS07/vzrepSbTjKzB29JBxrV9MqiCuid/Hf7B9NE/MgY53neqXdpoLePzCB5fMAD4q6ZF5W+1q7/GIvI75s=; AWSALBTGCORS=Snji2K/Av0g2zgKkZCj6slZu5nTpYvwBEhJn/IaPMvqZhrLbeqmMVSFj1+u17O0sILgRtRITMXapEB00gKV1hVYzDhEu0H4mnS07/vzrepSbTjKzB29JBxrV9MqiCuid/Hf7B9NE/MgY53neqXdpoLePzCB5fMAD4q6ZF5W+1q7/GIvI75s='

headers = {
    'dmip-tenant-id': dmpipTenantId,
    'dmip-application-name': dmipApplicationName,
    'dmip-token': dmipToken,
    'Content-Type': contentType,
    'Cookie': cookie
}

###### Chamada no dmps

In [ ]:
listaResultadoRMAHardFilterSemFlex = execute_requests(listaResultadoPythonHardFilterSemFlex, url, headers)

###### Validação Resultados

In [ ]:
listaValidacao = []
# Primeira validação, verificar se tudo que foi requisitado voltou
if len(listaResultadoRMAHardFilterSemFlex) != len(listaResultadoPythonHardFilterSemFlex):
    print('Erro na quantidade de casos retornados')
# Caso tenha voltado a mesma quantidade de casos, verificar se os valores são iguais
else:
    # Percorremos o tamanho da lista de resultados do python, pois assim garantimos que todos nossos casos foram verificados
    for i in range(len(listaResultadoPythonHardFilterSemFlex)):
        # Flag será True, se qualquer comparação falhar temos um problema de resultado
        validacao = True 
        validacaoComErro = []
        #print(i)
        # Verifica saídas de decisão
        if listaResultadoRMAHardFilterSemFlex[i]['saida']['mensagemFiltro'] != listaResultadoPythonHardFilterSemFlex[i]['payloadHomol']['saidas']['mensagemFiltro']:
            validacao = False
            validacaoComErro.append(1)
        if listaResultadoRMAHardFilterSemFlex[i]['saida']['mensagemFiltroFinal'] != listaResultadoPythonHardFilterSemFlex[i]['payloadHomol']['saidas']['mensagemFinal']:
            validacao = False
            validacaoComErro.append(2)
        if listaResultadoRMAHardFilterSemFlex[i]['saida']['statusDecisao'] != listaResultadoPythonHardFilterSemFlex[i]['payloadHomol']['saidas']['statusDecisao']:
            validacao = False
            validacaoComErro.append(3)

        # Verifica Flags
        if listaResultadoRMAHardFilterSemFlex[i]['saida']['hasInvalidRegistration'] != listaResultadoPythonHardFilterSemFlex[i]['payloadHomol']['saidas']['hasInvalidRegistration']:
            validacao = False
            validacaoComErro.append(4)
        if listaResultadoRMAHardFilterSemFlex[i]['saida']['hasDeniedOngoingRegistration'] != listaResultadoPythonHardFilterSemFlex[i]['payloadHomol']['saidas']['hasDeniedOngoingRegistration']:
            validacao = False
            validacaoComErro.append(5)
        if listaResultadoRMAHardFilterSemFlex[i]['saida']['hasDeactivatedOrTestAccount'] != listaResultadoPythonHardFilterSemFlex[i]['payloadHomol']['saidas']['hasDeactivatedOrTestAccount']:
            validacao = False    
            validacaoComErro.append(6)        
        if listaResultadoRMAHardFilterSemFlex[i]['saida']['hasOutdatedOs'] != listaResultadoPythonHardFilterSemFlex[i]['payloadHomol']['saidas']['hasOutdatedOs']:
            validacao = False   
            validacaoComErro.append(7)
        if listaResultadoRMAHardFilterSemFlex[i]['saida']['isInvalidCpf'] != listaResultadoPythonHardFilterSemFlex[i]['payloadHomol']['saidas']['isInvalidCpf']:
            validacao = False   
            validacaoComErro.append(8)
        if listaResultadoRMAHardFilterSemFlex[i]['saida']['isDeadUser'] != listaResultadoPythonHardFilterSemFlex[i]['payloadHomol']['saidas']['isDeadUser']:
            validacao = False   
            validacaoComErro.append(9)
        if listaResultadoRMAHardFilterSemFlex[i]['saida']['isFraudPp'] != listaResultadoPythonHardFilterSemFlex[i]['payloadHomol']['saidas']['isFraudPP']:
            validacao = False  
            validacaoComErro.append(10)
        if listaResultadoRMAHardFilterSemFlex[i]['saida']['isChargebackPp'] != listaResultadoPythonHardFilterSemFlex[i]['payloadHomol']['saidas']['isChargebackPP']:
            validacao = False  
            validacaoComErro.append(11)
        if listaResultadoRMAHardFilterSemFlex[i]['saida']['isOutAgeRange'] != listaResultadoPythonHardFilterSemFlex[i]['payloadHomol']['saidas']['isOutAgeRange']:
            validacao = False  
            validacaoComErro.append(12)
        if listaResultadoRMAHardFilterSemFlex[i]['saida']['isDelayPp'] != listaResultadoPythonHardFilterSemFlex[i]['payloadHomol']['saidas']['isDelayPP']:
            validacao = False  
            validacaoComErro.append(13)
        if listaResultadoRMAHardFilterSemFlex[i]['saida']['isDelayOr'] != listaResultadoPythonHardFilterSemFlex[i]['payloadHomol']['saidas']['isDelayOr']:
            validacao = False  
            validacaoComErro.append(14)
        if listaResultadoRMAHardFilterSemFlex[i]['saida']['isRenegFnv'] != listaResultadoPythonHardFilterSemFlex[i]['payloadHomol']['saidas']['isRenegFNV']:
            validacao = False  
            validacaoComErro.append(15)
        if listaResultadoRMAHardFilterSemFlex[i]['saida']['hasRestrictionBvs'] != listaResultadoPythonHardFilterSemFlex[i]['payloadHomol']['saidas']['hasRestrictionBVS']:
            validacao = False 
            validacaoComErro.append(16)
        if listaResultadoRMAHardFilterSemFlex[i]['saida']['hasRestrictionSerasa'] != listaResultadoPythonHardFilterSemFlex[i]['payloadHomol']['saidas']['hasRestrictionSerasa']:
            validacao = False 
            validacaoComErro.append(17)
        if listaResultadoRMAHardFilterSemFlex[i]['saida']['isUserDesenrola'] != listaResultadoPythonHardFilterSemFlex[i]['payloadHomol']['saidas']['isUserDesenrola']:
            validacao = False 
            validacaoComErro.append(18)
        if listaResultadoRMAHardFilterSemFlex[i]['saida']['isNoHitBureaus'] != listaResultadoPythonHardFilterSemFlex[i]['payloadHomol']['saidas']['isNoHitBureau']:
            validacao = False 
            validacaoComErro.append(19)
        
        # Verifica se a próxima etapa está sendo aplicada
        if listaResultadoRMAHardFilterSemFlex[i]['saida']['statusDecisao'] == "NEGADO":
            if listaResultadoRMAHardFilterSemFlex[i]['saida']['etapa'] != "FIM":
                validacao = False
                validacaoComErro.append(20)
        if listaResultadoRMAHardFilterSemFlex[i]['saida']['statusDecisao'] == "APROVADO":
            if listaResultadoRMAHardFilterSemFlex[i]['saida']['etapa'] != "POLITICA CONCESSAO":
                validacao = False
                validacaoComErro.append(21)

        objetoValidacao = {
            'caso': i,
            'validacao': validacao,
            'debugValidacao': validacaoComErro,
            'request': listaResultadoPythonHardFilterSemFlex[i],
            'response': listaResultadoRMAHardFilterSemFlex[i]
        }

        listaValidacao.append(objetoValidacao)

countOK = 0
countNOK = 0
for validacao in listaValidacao:
    if validacao['validacao'] == True:
        countOK = countOK + 1
    else:
        countNOK = countNOK + 1
        if countNOK <= 10:
            print(f'Caso {validacao["caso"]} com erro')

print('Etapa Hard Filter - Não Flexibilizado')
print(f'Quantidade de cenários Totais:' ,len(listaValidacao))
print(f'Cenários Validados: {countOK} ')
print(f'Cenários com Divergência: {countNOK}')

Etapa Hard Filter - Não Flexibilizado
Quantidade de cenários Totais: 642
Cenários Validados: 642 
Cenários com Divergência: 0


###### Extrai um caso Aprovado e um Negado dessa etapa

In [ ]:
# Parametros: Objeto de massa, lista de resultados da etapa, nome da etapa ( string para sinalizar na massa ), retorno é um dataframe
cenariosHardFilter_Etapa2 = arquivoExemplos(massa_EndToEnd.get_massa_final(),listaResultadoPythonHardFilterSemFlex,'Hard Filter Nao Flexibilizado')
cenariosHardFilter_Etapa2

,solicitante.numeroDocumento,solicitante.consumerID,etapa,solicitante.qtdIF,solicitante.fonteRenda,solicitante.flagCadastroIrregular,solicitante.flagCadastroNegado,solicitante.flagCadastroEmAndamento,solicitante.flagContaInativa,solicitante.flagContaTeste,...,solicitante.flagCCRot,solicitante.flagInvestimentoPrincipalidade,solicitante.investimentoTotal,solicitante.saldoInvestimento,solicitante.listaRestritivosSerasa.0.tipo,solicitante.listaRestritivosSerasa.0.montanteTotal,solicitante.listaRestritivosSerasa.0.valorTotal,Debug - ResultadoEsperado,Debug - Cenario,Debug - idSolicitacao Massa
0,12345678909,101010101,,1,01. Fopag,0,0,0,0,0,...,0,1,0,0,N,0,0,APROVADO,Hard Filter Nao Flexibilizado,0
1,12345678909,10002,,1,01. Fopag,0,0,0,0,0,...,0,1,0,0,N,0,0,NEGADO,Hard Filter Nao Flexibilizado,4


### Chamada RMA -> POLITICA CONCESSAO + BACEN SEM FLEXIBILIZAÇÃO etapa = "POLITICA CONCESSAO" 

###### Remove os cenários recusados na etapa anterior, seta a etapa politica para a próxima execução e também transfere as saídas do RMA para a entrada para próxima execução

In [ ]:
listaCenariosPoliticaConcessao = copy.deepcopy(listaResultadoPythonHardFilterSemFlex)
listaCenariosPoliticaConcessaoAprovados = []
for i in range(len(listaCenariosPoliticaConcessao)):
    if listaCenariosPoliticaConcessao[i]['payloadHomol']['saidas']['statusDecisao'] == "APROVADO":
        listaCenariosPoliticaConcessao[i]['etapa'] = "POLITICA CONCESSAO"
        listaCenariosPoliticaConcessaoAprovados.append(listaCenariosPoliticaConcessao[i])

print(f'Quantidade de cenários totais após o filtro:', len(listaCenariosPoliticaConcessaoAprovados))

Quantidade de cenários totais após o filtro: 639


###### Respostas esperadas

In [ ]:
listaResultadoPythonPolConcessao = rp.execucaoModular(listaCenariosPoliticaConcessaoAprovados,rp.etapapoliticaConcessao)

In [ ]:
results = []
for caso in listaResultadoPythonPolConcessao:
    results.append(caso['payloadHomol']['saidas']['statusDecisao'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

,valor,Freq
0,APROVADO,379
1,NEGADO,260


In [ ]:
results = []
for caso in listaResultadoPythonPolConcessao:
    results.append(caso['payloadHomol']['intermediarias']['flagClienteExistente'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

,valor,Freq
0,1,126
1,0,513


In [ ]:
results = []
for caso in listaResultadoPythonPolConcessao:
    results.append(caso['payloadHomol']['saidas']['limiteFinal'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

,valor,Freq
0,10000,105
1,0,260
2,900,256
3,4000,18


###### Parâmetros da chamada no dmps

In [ ]:
url = 'https://fico-dmps-lb-1479769718.us-east-1.elb.amazonaws.com/dmip-gw/dmip/request/servicosinteste13/response/servicosoutteste133'

dmpipTenantId = '1'
dmipApplicationName = 'PicPayDev'
dmipToken = '44d23b71-d44d-4759-b438-f73117e4dfbe'
contentType = 'application/vnd.fico.dmip.v3.0+text;charset=UTF-8'
cookie = 'JSESSIONID=93EAF2878176A0803A72F1BB66DA7935; AWSALB=8RWW5Bzifqu2tibNntmXzHbDN/5wGt861nvJnNq5NqSDjAlWAm1WaiGyelJP2tiqapQZeFWkU1VkATs/Qyeza8F0PJLkuNb0pelK2/xO6rgMTIeGC+Gk01II2lWX; AWSALBCORS=8RWW5Bzifqu2tibNntmXzHbDN/5wGt861nvJnNq5NqSDjAlWAm1WaiGyelJP2tiqapQZeFWkU1VkATs/Qyeza8F0PJLkuNb0pelK2/xO6rgMTIeGC+Gk01II2lWX; AWSALBTG=Snji2K/Av0g2zgKkZCj6slZu5nTpYvwBEhJn/IaPMvqZhrLbeqmMVSFj1+u17O0sILgRtRITMXapEB00gKV1hVYzDhEu0H4mnS07/vzrepSbTjKzB29JBxrV9MqiCuid/Hf7B9NE/MgY53neqXdpoLePzCB5fMAD4q6ZF5W+1q7/GIvI75s=; AWSALBTGCORS=Snji2K/Av0g2zgKkZCj6slZu5nTpYvwBEhJn/IaPMvqZhrLbeqmMVSFj1+u17O0sILgRtRITMXapEB00gKV1hVYzDhEu0H4mnS07/vzrepSbTjKzB29JBxrV9MqiCuid/Hf7B9NE/MgY53neqXdpoLePzCB5fMAD4q6ZF5W+1q7/GIvI75s='

headers = {
    'dmip-tenant-id': dmpipTenantId,
    'dmip-application-name': dmipApplicationName,
    'dmip-token': dmipToken,
    'Content-Type': contentType,
    'Cookie': cookie
}

###### Chamada no dmps

In [ ]:
listaResultadoRMAPoliticaConcessao = execute_requests(listaResultadoPythonPolConcessao, url, headers)

###### Validação Resultados

In [ ]:
listaValidacao = []
# Primeira validação, verificar se tudo que foi requisitado voltou
if len(listaResultadoRMAPoliticaConcessao) != len(listaResultadoPythonPolConcessao):
    print('Erro na quantidade de casos retornados')
# Caso tenha voltado a mesma quantidade de casos, verificar se os valores são iguais
else:
    # Percorremos o tamanho da lista de resultados do python, pois assim garantimos que todos nossos casos foram verificados
    for i in range(len(listaResultadoPythonPolConcessao)):
        #print(i)
        # Flag será True, se qualquer comparação falhar temos um problema de resultado
        validacao = True 
        validacaoComErro = []


        # Verifica Limite
        if listaResultadoRMAPoliticaConcessao[i]['saida']['limite'] != listaResultadoPythonPolConcessao[i]['payloadHomol']['saidas']['limiteFinal']:
            validacao = False
            validacaoComErro.append(1)

        # Verifica statusDecisao
        if listaResultadoRMAPoliticaConcessao[i]['saida']['statusDecisao'] != listaResultadoPythonPolConcessao[i]['payloadHomol']['saidas']['statusDecisao']:
            validacao = False
            validacaoComErro.append(2)
        # Faremos a verificação de motivos apenas se o statusDecisão estiver OK, caso contrário já existe a divergência por statusDecisao.
        else:
            # apenas casos negados vai existir o vetor de regras negativas, portanto fazemos a verificação apenas para esses casos.
            if (listaResultadoRMAPoliticaConcessao[i]['saida']['statusDecisao'] == "NEGADO" or 
                listaResultadoPythonPolConcessao[i]['payloadHomol']['saidas']['statusDecisao'] == "NEGADO"):
                # Verifica se existe lista de negativas nos dois lados
                if (('regrasNegativas' in listaResultadoRMAPoliticaConcessao[i]['saida'] and 
                    len(listaResultadoPythonPolConcessao[i]['payloadHomol']['saidas']['regrasNegativas']) == 0
                    ) 
                    or
                    (
                    'regrasNegativas' not in listaResultadoRMAPoliticaConcessao[i]['saida'] and 
                     len(listaResultadoPythonPolConcessao[i]['payloadHomol']['saidas']['regrasNegativas']) != 0
                    )
                   ):
                    validacao = False
                    validacaoComErro.append(3)
                else:
                    # Se existir a lista de negativas nos dois lados, verificamos se os motivos são iguais
                    if ('regrasNegativas' in listaResultadoRMAPoliticaConcessao[i]['saida'] and 
                        len(listaResultadoPythonPolConcessao[i]['payloadHomol']['saidas']['regrasNegativas']) > 0):
                        # Verifica se a quantidade de motivos é igual
                        if len(listaResultadoRMAPoliticaConcessao[i]['saida']['regrasNegativas']) != len(listaResultadoPythonPolConcessao[i]['payloadHomol']['saidas']['regrasNegativas']):
                            validacao = False
                            validacaoComErro.append(4)
                        # Se tamanho igual, varre cada motivo dos dois lados e compara
                        else:
                            for j in range(len(listaResultadoRMAPoliticaConcessao[i]['saida']['regrasNegativas'])):
                                if listaResultadoRMAPoliticaConcessao[i]['saida']['regrasNegativas'][j]['descRegra'] != listaResultadoPythonPolConcessao[i]['payloadHomol']['saidas']['regrasNegativas'][j]['descricao']:
                                    validacao = False
                                    validacaoComErro.append(5)
                                if listaResultadoRMAPoliticaConcessao[i]['saida']['regrasNegativas'][j]['nomeRegra'] != listaResultadoPythonPolConcessao[i]['payloadHomol']['saidas']['regrasNegativas'][j]['nomeRegra']:
                                    validacao = False
                                    validacaoComErro.append(6)
            
        # Verifica se a próxima etapa está sendo aplicada
        if listaResultadoRMAPoliticaConcessao[i]['saida']['statusDecisao'] == "NEGADO":
            if listaResultadoRMAPoliticaConcessao[i]['saida']['etapa'] != "FIM":
                validacao = False
                validacaoComErro.append(7)
        if listaResultadoRMAPoliticaConcessao[i]['saida']['statusDecisao'] == "APROVADO":
            if listaResultadoRMAPoliticaConcessao[i]['saida']['etapa'] != "BUREAU CADASTRAL":
                validacao = False
                validacaoComErro.append(8)

        objetoValidacao = {
            'caso': i,
            'validacao': validacao,
            'debugValidacao': validacaoComErro,
            'request': listaResultadoPythonPolConcessao[i],
            'response': listaResultadoRMAPoliticaConcessao[i]
        }

        listaValidacao.append(objetoValidacao)

countOK = 0
countNOK = 0
for validacao in listaValidacao:
    if validacao['validacao'] == True:
        countOK = countOK + 1
    else:
        countNOK = countNOK + 1
        if countNOK <= 5:
            print(f'Caso {validacao["caso"]} com erro')
            print(f'flagsDebug: {validacao["debugValidacao"]}')
            print('------------------------------------------')

print('Etapa Politica Concessao + Bacen Não Flexibilizado')
print(f'Quantidade de cenários Totais:' ,len(listaValidacao))
print(f'Cenários Validados: {countOK} ')
print(f'Cenários com Divergência: {countNOK}')

Etapa Politica Concessao + Bacen Não Flexibilizado
Quantidade de cenários Totais: 639
Cenários Validados: 639 
Cenários com Divergência: 0


###### Extrai um caso Aprovado e um Negado dessa etapa

In [ ]:
# Parametros: Objeto de massa, lista de resultados da etapa, nome da etapa ( string para sinalizar na massa ), retorno é um dataframe
cenariosPoliticaConcessaoBacenSemFlex = arquivoExemplos(massa_EndToEnd.get_massa_final(),listaResultadoPythonPolConcessao,'Politica Concessao + Bacen Sem Flexibilizacao')
cenariosPoliticaConcessaoBacenSemFlex

,solicitante.numeroDocumento,solicitante.consumerID,etapa,solicitante.qtdIF,solicitante.fonteRenda,solicitante.flagCadastroIrregular,solicitante.flagCadastroNegado,solicitante.flagCadastroEmAndamento,solicitante.flagContaInativa,solicitante.flagContaTeste,...,solicitante.flagCCRot,solicitante.flagInvestimentoPrincipalidade,solicitante.investimentoTotal,solicitante.saldoInvestimento,solicitante.listaRestritivosSerasa.0.tipo,solicitante.listaRestritivosSerasa.0.montanteTotal,solicitante.listaRestritivosSerasa.0.valorTotal,Debug - ResultadoEsperado,Debug - Cenario,Debug - idSolicitacao Massa
0,12345678909,101010101,,1,01. Fopag,0,0,0,0,0,...,0,1,0,0,N,0,0,APROVADO,Politica Concessao + Bacen Sem Flexibilizacao,0
1,12345678909,9,,1,01. Fopag,0,0,0,0,0,...,0,1,0,0,N,0,0,NEGADO,Politica Concessao + Bacen Sem Flexibilizacao,50


### Chamada RMA -> BUREAU CADASTRAL etapa = "BUREAU CADASTRAL" 

###### Remove os cenários recusados na etapa anterior, seta a etapa politica para a próxima execução e também transfere as saídas do RMA para a entrada para próxima execução

In [ ]:
listaCenariosPoliticaBureauCadastral = copy.deepcopy(listaResultadoPythonPolConcessao)
listaCenariosPoliticaBureauCadastralAprovados = []
for i in range(len(listaCenariosPoliticaBureauCadastral)):
    if listaCenariosPoliticaBureauCadastral[i]['payloadHomol']['saidas']['statusDecisao'] == "APROVADO":
        listaCenariosPoliticaBureauCadastral[i]['etapa'] = "BUREAU CADASTRAL"
        listaCenariosPoliticaBureauCadastral[i]['limitePoliticaConcessao'] = listaResultadoRMAPoliticaConcessao[i]['saida']['limite']
        listaCenariosPoliticaBureauCadastralAprovados.append(listaCenariosPoliticaBureauCadastral[i])

print(f'Quantidade de cenários totais após o filtro:', len(listaCenariosPoliticaBureauCadastralAprovados))

Quantidade de cenários totais após o filtro: 379


###### Respostas esperadas

In [ ]:
listaResultadoPythonBureauCadastral = rp.execucaoModular(listaCenariosPoliticaBureauCadastralAprovados,rp.etapaBureauCadastral)

In [ ]:
results = []
for caso in listaResultadoPythonBureauCadastral:
    results.append(caso['marcacaoMesa'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

,valor,Freq
0,N,319
1,MESA,60


In [ ]:
results = []
for caso in listaResultadoPythonBureauCadastral:
    results.append(caso['payloadHomol']['saidas']['statusDecisao'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

,valor,Freq
0,APROVADO,370
1,NEGADO,3
2,PENDENTE,6


###### Parâmetros da chamada no dmps

In [ ]:
url = 'https://fico-dmps-lb-1479769718.us-east-1.elb.amazonaws.com/dmip-gw/dmip/request/servicosinteste13/response/servicosoutteste133'

dmpipTenantId = '1'
dmipApplicationName = 'PicPayDev'
dmipToken = '44d23b71-d44d-4759-b438-f73117e4dfbe'
contentType = 'application/vnd.fico.dmip.v3.0+text;charset=UTF-8'
cookie = 'JSESSIONID=93EAF2878176A0803A72F1BB66DA7935; AWSALB=8RWW5Bzifqu2tibNntmXzHbDN/5wGt861nvJnNq5NqSDjAlWAm1WaiGyelJP2tiqapQZeFWkU1VkATs/Qyeza8F0PJLkuNb0pelK2/xO6rgMTIeGC+Gk01II2lWX; AWSALBCORS=8RWW5Bzifqu2tibNntmXzHbDN/5wGt861nvJnNq5NqSDjAlWAm1WaiGyelJP2tiqapQZeFWkU1VkATs/Qyeza8F0PJLkuNb0pelK2/xO6rgMTIeGC+Gk01II2lWX; AWSALBTG=Snji2K/Av0g2zgKkZCj6slZu5nTpYvwBEhJn/IaPMvqZhrLbeqmMVSFj1+u17O0sILgRtRITMXapEB00gKV1hVYzDhEu0H4mnS07/vzrepSbTjKzB29JBxrV9MqiCuid/Hf7B9NE/MgY53neqXdpoLePzCB5fMAD4q6ZF5W+1q7/GIvI75s=; AWSALBTGCORS=Snji2K/Av0g2zgKkZCj6slZu5nTpYvwBEhJn/IaPMvqZhrLbeqmMVSFj1+u17O0sILgRtRITMXapEB00gKV1hVYzDhEu0H4mnS07/vzrepSbTjKzB29JBxrV9MqiCuid/Hf7B9NE/MgY53neqXdpoLePzCB5fMAD4q6ZF5W+1q7/GIvI75s='

headers = {
    'dmip-tenant-id': dmpipTenantId,
    'dmip-application-name': dmipApplicationName,
    'dmip-token': dmipToken,
    'Content-Type': contentType,
    'Cookie': cookie
}

###### Chamada no dmps

In [ ]:
listaResultadoRMABureauCadastral = execute_requests(listaResultadoPythonBureauCadastral, url, headers)

###### Validação Resultados

In [ ]:
listaValidacao = []
# Primeira validação, verificar se tudo que foi requisitado voltou
if len(listaResultadoRMABureauCadastral) != len(listaResultadoPythonBureauCadastral):
    print('Erro na quantidade de casos retornados')
# Caso tenha voltado a mesma quantidade de casos, verificar se os valores são iguais
else:
    # Percorremos o tamanho da lista de resultados do python, pois assim garantimos que todos nossos casos foram verificados
    for i in range(len(listaResultadoPythonBureauCadastral)):
        #print(i)
        # Flag será True, se qualquer comparação falhar temos um problema de resultado
        validacao = True 
        validacaoComErro = []

        # Verifica Limite
        #if listaResultadoRMABureauCadastral[i]['saida']['limite'] != listaResultadoPythonBureauCadastral[i]['payloadHomol']['saidas']['limiteFinal']:
        #    validacao = False
        #    validacaoComErro.append(0)

        # Verifica statusDecisao
        if listaResultadoRMABureauCadastral[i]['saida']['statusDecisao'] != listaResultadoPythonBureauCadastral[i]['payloadHomol']['saidas']['statusDecisao']:
            validacao = False
            validacaoComErro.append(1)
        # Faremos a verificação de motivos apenas se o statusDecisão estiver OK, caso contrário já existe a divergência por statusDecisao.
        else:
            # apenas casos negados vai existir o vetor de regras negativas, portanto fazemos a verificação apenas para esses casos.
            if (listaResultadoRMABureauCadastral[i]['saida']['statusDecisao'] == "NEGADO" or 
                listaResultadoPythonBureauCadastral[i]['payloadHomol']['saidas']['statusDecisao'] == "NEGADO"):
                # Verifica se existe lista de negativas nos dois lados
                if (('regrasNegativas' in listaResultadoRMABureauCadastral[i]['saida'] and 
                    len(listaResultadoPythonBureauCadastral[i]['payloadHomol']['saidas']['regrasNegativas']) == 0
                    ) 
                    or
                    (
                    'regrasNegativas' not in listaResultadoRMABureauCadastral[i]['saida'] and 
                     len(listaResultadoPythonBureauCadastral[i]['payloadHomol']['saidas']['regrasNegativas']) != 0
                    )
                   ):
                    validacao = False
                    validacaoComErro.append(2)
                else:
                    # Se existir a lista de negativas nos dois lados, verificamos se os motivos são iguais
                    if ('regrasNegativas' in listaResultadoRMABureauCadastral[i]['saida'] and 
                        len(listaResultadoPythonBureauCadastral[i]['payloadHomol']['saidas']['regrasNegativas']) > 0):
                        # Verifica se a quantidade de motivos é igual
                        if len(listaResultadoRMABureauCadastral[i]['saida']['regrasNegativas']) != len(listaResultadoPythonBureauCadastral[i]['payloadHomol']['saidas']['regrasNegativas']):
                            validacao = False
                            validacaoComErro.append(3)
                        # Se tamanho igual, varre cada motivo dos dois lados e compara
                        else:
                            for j in range(len(listaResultadoRMABureauCadastral[i]['saida']['regrasNegativas'])):
                                if listaResultadoRMABureauCadastral[i]['saida']['regrasNegativas'][j]['descRegra'] != listaResultadoPythonBureauCadastral[i]['payloadHomol']['saidas']['regrasNegativas'][j]['descricao']:
                                    validacao = False
                                    validacaoComErro.append(4)
                                if listaResultadoRMABureauCadastral[i]['saida']['regrasNegativas'][j]['nomeRegra'] != listaResultadoPythonBureauCadastral[i]['payloadHomol']['saidas']['regrasNegativas'][j]['nomeRegra']:
                                    validacao = False
                                    validacaoComErro.append(5)
            
        # Verifica se a próxima etapa está sendo aplicada
        if listaResultadoRMABureauCadastral[i]['saida']['statusDecisao'] == "NEGADO":
            if listaResultadoRMABureauCadastral[i]['saida']['etapa'] != "FIM":
                validacao = False
                validacaoComErro.append(6)
        if listaResultadoRMABureauCadastral[i]['saida']['statusDecisao'] == "APROVADO":
            if listaResultadoRMABureauCadastral[i]['saida']['etapa'] != "SERASA":
                validacao = False
                validacaoComErro.append(7)

        objetoValidacao = {
            'caso': i,
            'validacao': validacao,
            'debugValidacao': validacaoComErro,
            'request': listaResultadoPythonBureauCadastral[i],
            'response': listaResultadoRMABureauCadastral[i]
        }

        listaValidacao.append(objetoValidacao)

countOK = 0
countNOK = 0
for validacao in listaValidacao:
    if validacao['validacao'] == True:
        countOK = countOK + 1
    else:
        countNOK = countNOK + 1
        if countNOK <= 5:
            print(f'Caso {validacao["caso"]} com erro')
            print(f'flagsDebug: {validacao["debugValidacao"]}')
            print('------------------------------------------')

print('Etapa Bureau Cadastral')
print(f'Quantidade de cenários Totais:' ,len(listaValidacao))
print(f'Cenários Validados: {countOK} ')
print(f'Cenários com Divergência: {countNOK}')

Etapa Bureau Cadastral
Quantidade de cenários Totais: 379
Cenários Validados: 379 
Cenários com Divergência: 0


###### Extrai um caso Aprovado e um Negado dessa etapa

In [ ]:
# Parametros: Objeto de massa, lista de resultados da etapa, nome da etapa ( string para sinalizar na massa ), retorno é um dataframe
cenariosBureauCadastral = arquivoExemplos(massa_EndToEnd.get_massa_final(),listaResultadoPythonBureauCadastral,'Bureau Cadastral')
cenariosBureauCadastral

,solicitante.numeroDocumento,solicitante.consumerID,etapa,solicitante.qtdIF,solicitante.fonteRenda,solicitante.flagCadastroIrregular,solicitante.flagCadastroNegado,solicitante.flagCadastroEmAndamento,solicitante.flagContaInativa,solicitante.flagContaTeste,...,solicitante.flagCCRot,solicitante.flagInvestimentoPrincipalidade,solicitante.investimentoTotal,solicitante.saldoInvestimento,solicitante.listaRestritivosSerasa.0.tipo,solicitante.listaRestritivosSerasa.0.montanteTotal,solicitante.listaRestritivosSerasa.0.valorTotal,Debug - ResultadoEsperado,Debug - Cenario,Debug - idSolicitacao Massa
0,12345678909,101010101,,1,01. Fopag,0,0,0,0,0,...,0,1,0,0,N,0,0,APROVADO,Bureau Cadastral,0
1,12345678909,12,,1,01. Fopag,0,0,0,0,0,...,0,1,0,0,N,0,0,NEGADO,Bureau Cadastral,576


### Chamada RMA -> SERASA etapa = "SERASA" 

###### Remove os cenários recusados na etapa anterior, seta a etapa politica para a próxima execução e também transfere as saídas do RMA para a entrada para próxima execução

In [ ]:
listaCenariosPoliticaSerasa = copy.deepcopy(listaResultadoPythonBureauCadastral)
listaCenariosPoliticaSerasaAprovados = []
for i in range(len(listaCenariosPoliticaSerasa)):
    if listaCenariosPoliticaSerasa[i]['payloadHomol']['saidas']['statusDecisao'] == "APROVADO":
        listaCenariosPoliticaSerasa[i]['etapa'] = "SERASA"
        listaCenariosPoliticaBureauCadastral[i]['limitePoliticaConcessao'] = listaResultadoRMAPoliticaConcessao[i]['saida']['limite']
        listaCenariosPoliticaSerasaAprovados.append(listaCenariosPoliticaSerasa[i])

print(f'Quantidade de cenários totais após o filtro:', len(listaCenariosPoliticaSerasaAprovados))

Quantidade de cenários totais após o filtro: 370


###### Respostas esperadas

In [ ]:
listaResultadoPythonetapaSerasa = rp.execucaoModular(listaCenariosPoliticaSerasaAprovados,rp.etapaSerasa)

In [ ]:
results = []
for caso in listaResultadoPythonetapaSerasa:
    results.append(caso['marcacaoMesa'])

freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

,valor,Freq
0,N,310
1,MESA,60


In [ ]:
results = []
for caso in listaResultadoPythonetapaSerasa:
    results.append(caso['payloadHomol']['saidas']['statusDecisao'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

,valor,Freq
0,APROVADO,353
1,NEGADO,16
2,PENDENTE,1


###### Parâmetros da chamada no dmps

In [ ]:
url = 'https://fico-dmps-lb-1479769718.us-east-1.elb.amazonaws.com/dmip-gw/dmip/request/servicosinteste13/response/servicosoutteste133'

dmpipTenantId = '1'
dmipApplicationName = 'PicPayDev'
dmipToken = '44d23b71-d44d-4759-b438-f73117e4dfbe'
contentType = 'application/vnd.fico.dmip.v3.0+text;charset=UTF-8'
cookie = 'JSESSIONID=93EAF2878176A0803A72F1BB66DA7935; AWSALB=8RWW5Bzifqu2tibNntmXzHbDN/5wGt861nvJnNq5NqSDjAlWAm1WaiGyelJP2tiqapQZeFWkU1VkATs/Qyeza8F0PJLkuNb0pelK2/xO6rgMTIeGC+Gk01II2lWX; AWSALBCORS=8RWW5Bzifqu2tibNntmXzHbDN/5wGt861nvJnNq5NqSDjAlWAm1WaiGyelJP2tiqapQZeFWkU1VkATs/Qyeza8F0PJLkuNb0pelK2/xO6rgMTIeGC+Gk01II2lWX; AWSALBTG=Snji2K/Av0g2zgKkZCj6slZu5nTpYvwBEhJn/IaPMvqZhrLbeqmMVSFj1+u17O0sILgRtRITMXapEB00gKV1hVYzDhEu0H4mnS07/vzrepSbTjKzB29JBxrV9MqiCuid/Hf7B9NE/MgY53neqXdpoLePzCB5fMAD4q6ZF5W+1q7/GIvI75s=; AWSALBTGCORS=Snji2K/Av0g2zgKkZCj6slZu5nTpYvwBEhJn/IaPMvqZhrLbeqmMVSFj1+u17O0sILgRtRITMXapEB00gKV1hVYzDhEu0H4mnS07/vzrepSbTjKzB29JBxrV9MqiCuid/Hf7B9NE/MgY53neqXdpoLePzCB5fMAD4q6ZF5W+1q7/GIvI75s='

headers = {
    'dmip-tenant-id': dmpipTenantId,
    'dmip-application-name': dmipApplicationName,
    'dmip-token': dmipToken,
    'Content-Type': contentType,
    'Cookie': cookie
}

###### Chamada no dmps

In [ ]:
listaResultadoRMAetapaSerasa = execute_requests(listaResultadoPythonetapaSerasa, url, headers)

###### Validação Resultados

In [ ]:
listaValidacao = []
# Primeira validação, verificar se tudo que foi requisitado voltou
if len(listaResultadoRMAetapaSerasa) != len(listaResultadoPythonetapaSerasa):
    print('Erro na quantidade de casos retornados')
# Caso tenha voltado a mesma quantidade de casos, verificar se os valores são iguais
else:
    # Percorremos o tamanho da lista de resultados do python, pois assim garantimos que todos nossos casos foram verificados
    for i in range(len(listaResultadoPythonetapaSerasa)):
        #print(i)
        # Flag será True, se qualquer comparação falhar temos um problema de resultado
        validacao = True 
        validacaoComErro = []

        # Verifica Flag Small Limit
        if listaResultadoRMAetapaSerasa[i]['saida']['flagSmallLimit'] != listaResultadoPythonetapaSerasa[i]['payloadHomol']['saidas']['flagSmallLimit']:
            validacao = False
            validacaoComErro.append(1)

        # Verifica Limite
        if listaResultadoRMAetapaSerasa[i]['saida']['limite'] != listaResultadoPythonetapaSerasa[i]['payloadHomol']['saidas']['limiteFinal']:
            validacao = False
            validacaoComErro.append(2)

        # Verifica Limite
        if listaResultadoRMAetapaSerasa[i]['saida']['vlrLimiteAprovado'] != listaResultadoPythonetapaSerasa[i]['payloadHomol']['saidas']['vlrLimiteAprovado']:
            validacao = False
            validacaoComErro.append(3)

        # Verifica statusDecisao
        if listaResultadoRMAetapaSerasa[i]['saida']['statusDecisao'] != listaResultadoPythonetapaSerasa[i]['payloadHomol']['saidas']['statusDecisao']:
            validacao = False
            validacaoComErro.append(4)
        # Faremos a verificação de motivos apenas se o statusDecisão estiver OK, caso contrário já existe a divergência por statusDecisao.
        else:
            # apenas casos negados vai existir o vetor de regras negativas, portanto fazemos a verificação apenas para esses casos.
            if (listaResultadoRMAetapaSerasa[i]['saida']['statusDecisao'] == "NEGADO" or 
                listaResultadoPythonetapaSerasa[i]['payloadHomol']['saidas']['statusDecisao'] == "NEGADO"):
                # Verifica se existe lista de negativas nos dois lados
                if (('regrasNegativas' in listaResultadoRMAetapaSerasa[i]['saida'] and 
                    len(listaResultadoPythonetapaSerasa[i]['payloadHomol']['saidas']['regrasNegativas']) == 0
                    ) 
                    or
                    (
                    'regrasNegativas' not in listaResultadoRMAetapaSerasa[i]['saida'] and 
                     len(listaResultadoPythonetapaSerasa[i]['payloadHomol']['saidas']['regrasNegativas']) != 0
                    )
                   ):
                    validacao = False
                    validacaoComErro.append(5)
                else:
                    # Se existir a lista de negativas nos dois lados, verificamos se os motivos são iguais
                    if ('regrasNegativas' in listaResultadoRMAetapaSerasa[i]['saida'] and 
                        len(listaResultadoPythonetapaSerasa[i]['payloadHomol']['saidas']['regrasNegativas']) > 0):
                        # Verifica se a quantidade de motivos é igual
                        if len(listaResultadoRMAetapaSerasa[i]['saida']['regrasNegativas']) != len(listaResultadoPythonetapaSerasa[i]['payloadHomol']['saidas']['regrasNegativas']):
                            validacao = False
                            validacaoComErro.append(6)
                        # Se tamanho igual, varre cada motivo dos dois lados e compara
                        else:
                            for j in range(len(listaResultadoRMAetapaSerasa[i]['saida']['regrasNegativas'])):
                                if listaResultadoRMAetapaSerasa[i]['saida']['regrasNegativas'][j]['descRegra'] != listaResultadoPythonetapaSerasa[i]['payloadHomol']['saidas']['regrasNegativas'][j]['descricao']:
                                    validacao = False
                                    validacaoComErro.append(7)
                                if listaResultadoRMAetapaSerasa[i]['saida']['regrasNegativas'][j]['nomeRegra'] != listaResultadoPythonetapaSerasa[i]['payloadHomol']['saidas']['regrasNegativas'][j]['nomeRegra']:
                                    validacao = False
                                    validacaoComErro.append(8)
            
        # Verifica se a próxima etapa está sendo aplicada
        if listaResultadoRMAetapaSerasa[i]['saida']['statusDecisao'] == "NEGADO":
            if listaResultadoRMAetapaSerasa[i]['saida']['etapa'] != "FIM":
                validacao = False
                validacaoComErro.append(9)
        if listaResultadoRMAetapaSerasa[i]['saida']['statusDecisao'] == "APROVADO":
            if listaResultadoRMAetapaSerasa[i]['saida']['etapa'] != "FIM":
                validacao = False
                validacaoComErro.append(10)

        objetoValidacao = {
            'caso': i,
            'validacao': validacao,
            'debugValidacao': validacaoComErro,
            'request': listaResultadoPythonetapaSerasa[i],
            'response': listaResultadoRMAetapaSerasa[i]
        }

        listaValidacao.append(objetoValidacao)

countOK = 0
countNOK = 0
for validacao in listaValidacao:
    if validacao['validacao'] == True:
        countOK = countOK + 1
    else:
        countNOK = countNOK + 1
        if countNOK <= 5:
            print(f'Caso {validacao["caso"]} com erro')
            print(f'flagsDebug: {validacao["debugValidacao"]}')
            print('------------------------------------------')

print('Etapa Serasa')
print(f'Quantidade de cenários Totais:' ,len(listaValidacao))
print(f'Cenários Validados: {countOK}')
print(f'Cenários com Divergência: {countNOK}')

Etapa Serasa
Quantidade de cenários Totais: 370
Cenários Validados: 370
Cenários com Divergência: 0


###### Extrai um caso Aprovado e um Negado dessa etapa

In [ ]:
#cenariosSerasa = arquivoExemplos(massa_EndToEnd.get_massa_final(),listaResultadoPythonetapaSerasa,'Serasa')

# Inicia os ids com -1, caso não encontre cenários aprovados e negados
idSolicitacaoAprovado = -1
idSolicitacaoNegado = -1
# Busca um id aprovado
for payload in listaResultadoPythonetapaSerasa:
    if payload['payloadHomol']['saidas']['statusDecisao'] == 'APROVADO':
        idSolicitacaoAprovado = payload['idSolicitacao']
        break
# Busca um id negado com um documento especifico
for payload in listaResultadoPythonetapaSerasa:
    if payload['payloadHomol']['saidas']['statusDecisao'] == 'NEGADO' and payload['solicitante']['numeroDocumento'] == '11111199111':
        idSolicitacaoNegado = payload['idSolicitacao']
        break
if idSolicitacaoAprovado == -1 or idSolicitacaoNegado == -1:
    if idSolicitacaoAprovado == -1:
        print('Não foi possível encontrar um cenário aprovado')
    if idSolicitacaoNegado == -1:
        print('Não foi possível encontrar um cenário negado')
else:
    # partindo do principio que o idSolicitacao no json é gerado a partir do indice da massa no sherlock
    aprovado = massa_EndToEnd.get_massa_final().iloc[[idSolicitacaoAprovado]].copy()
    aprovado['Debug - ResultadoEsperado'] = 'APROVADO'
    recusado = massa_EndToEnd.get_massa_final().iloc[[idSolicitacaoNegado]].copy()
    recusado['Debug - ResultadoEsperado'] = 'NEGADO'
    cenariosSerasa = pd.concat([aprovado, recusado], axis=0)
    cenariosSerasa['Debug - Cenario'] = 'Serasa'
    cenariosSerasa['Debug - idSolicitacao Massa'] = cenariosSerasa.index
    cenariosSerasa.reset_index(drop=True, inplace=True)




cenariosSerasa

,solicitante.numeroDocumento,solicitante.consumerID,etapa,solicitante.qtdIF,solicitante.fonteRenda,solicitante.flagCadastroIrregular,solicitante.flagCadastroNegado,solicitante.flagCadastroEmAndamento,solicitante.flagContaInativa,solicitante.flagContaTeste,...,solicitante.flagCCRot,solicitante.flagInvestimentoPrincipalidade,solicitante.investimentoTotal,solicitante.saldoInvestimento,solicitante.listaRestritivosSerasa.0.tipo,solicitante.listaRestritivosSerasa.0.montanteTotal,solicitante.listaRestritivosSerasa.0.valorTotal,Debug - ResultadoEsperado,Debug - Cenario,Debug - idSolicitacao Massa
0,12345678909,101010101,,1,01. Fopag,0,0,0,0,0,...,0,1,0,0,N,0,0,APROVADO,Serasa,0
1,11111199111,13,,0,01. Fopag,0,0,0,0,0,...,0,1,0,0,N,0,0,NEGADO,Serasa,589


### Arquivo final para testes integrados

###### Recupera cenários especificos

In [ ]:
# Inicia os ids com -1, caso não encontre cenários aprovados e negados
idSolicitacaoRendaBatch = -1
idSolicitacaoRendaOPF = -1
idSolicitacaoRendaOPFServidor = -1

# Caso de renda BATCH
for payload in listaResultadoPythonetapaSerasa:
    if (payload['payloadHomol']['saidas']['flagTipoRenda'] == 'BATCH' and payload['payloadHomol']['saidas']['statusDecisao'] == 'APROVADO'):
        idSolicitacaoRendaBatch = payload['idSolicitacao']
        statusDecisaoBatch = payload['payloadHomol']['saidas']['statusDecisao']
        break
# Caso de renda OPF
for payload in listaResultadoPythonetapaSerasa:
    if (payload['payloadHomol']['saidas']['flagTipoRenda'] == 'OPEN FINANCE' and payload['payloadHomol']['saidas']['statusDecisao'] == 'APROVADO'):
        idSolicitacaoRendaOPF = payload['idSolicitacao']
        statusDecisaoOPF = payload['payloadHomol']['saidas']['statusDecisao']
        break
# Caso de renda SERVIDOR
for payload in listaResultadoPythonetapaSerasa:
    if (payload['payloadHomol']['saidas']['flagTipoRenda'] == 'OPEN FINANCE SERVIDOR' and payload['payloadHomol']['saidas']['statusDecisao'] == 'APROVADO'):
        idSolicitacaoRendaOPFServidor = payload['idSolicitacao']
        statusDecisaoOPFServidor = payload['payloadHomol']['saidas']['statusDecisao']
        break

if idSolicitacaoRendaBatch == -1 or idSolicitacaoRendaOPF == -1 or idSolicitacaoRendaOPFServidor == -1:
    print('Não foi possível encontrar todos cenários de renda')
else:
    # partindo do principio que o idSolicitacao no json é gerado a partir do indice da massa no sherlock
    casoBatch = massa_EndToEnd.get_massa_final().iloc[[idSolicitacaoRendaBatch]].copy()
    casoBatch['Debug - ResultadoEsperado'] = statusDecisaoBatch
    casoBatch['Debug - Cenario'] = 'RENDA BATCH'
    casoBatch['Debug - idSolicitacao Massa'] = idSolicitacaoRendaBatch

    casoOPF = massa_EndToEnd.get_massa_final().iloc[[idSolicitacaoRendaOPF]].copy()
    casoOPF['Debug - ResultadoEsperado'] = statusDecisaoOPF
    casoOPF['Debug - Cenario'] = 'RENDA OPF'
    casoOPF['Debug - idSolicitacao Massa'] = idSolicitacaoRendaOPF

    casoOPFServidor = massa_EndToEnd.get_massa_final().iloc[[idSolicitacaoRendaOPFServidor]].copy()
    casoOPFServidor['Debug - ResultadoEsperado'] = statusDecisaoOPFServidor
    casoOPFServidor['Debug - Cenario'] = 'RENDA OPF SERVIDOR'
    casoOPFServidor['Debug - idSolicitacao Massa'] = idSolicitacaoRendaOPFServidor

    cenariosRenda = pd.concat([casoBatch, casoOPF,casoOPFServidor], axis=0)
    cenariosRenda.reset_index(drop=True, inplace=True)
    cenariosRenda

Não foi possível encontrar todos cenários de renda


###### Dataframe Final com todos cenários para o teste integrado

In [ ]:
cenariosTesteIntegrado = pd.concat([cenariosHardFilter_Etapa1,
                                    cenariosFiltrosPolitica,
                                    cenariosBacenFlexibilizado,
                                    cenariosHardFilter_Etapa2,
                                    cenariosPoliticaConcessaoBacenSemFlex,
                                    cenariosBureauCadastral,
                                    cenariosSerasa
                                    #,cenariosRenda
                                    ], 
                                            axis=0)
cenariosTesteIntegrado.reset_index(drop=True, inplace=True)

In [ ]:
cenariosTesteIntegrado
cenariosTesteIntegrado.to_csv(r'massas/cenariosTesteIntegrado.csv',index=False)

###### Lista campos

In [ ]:
columns_list = cenariosTesteIntegrado.columns.tolist()

for element in columns_list:
    print(element)


###### Tratamentos finais e export

In [ ]:
cenariosTesteIntegradoRenomeado = cenariosTesteIntegrado
cenariosTesteIntegradoRenomeado.rename(columns = {
        'solicitante.flagCadastroIrregular': 'has_invalid_contact_information',
        'solicitante.flagOSDefasado': 'has_outdated_os',
        'solicitante.flagContaInativa': 'has_deactivated_account',
        'solicitante.flagContaTeste': 'is_picpay_test_account',
        'solicitante.flagCadastroNegado': 'has_denied_registration',
        'solicitante.flagCadastroEmAndamento': 'has_ongoing_registration',
        'solicitante.flagFraudePicpay': 'is_user_fraud_pp',
        'solicitante.flagFraudeCartaoPicpay': 'is_ppcard_fraud',
        'solicitante.flagBloqueados': 'is_in_blck_ppcard_reneg',
        'solicitante.flagHoldForced': 'is_on_hold_forced',
        'solicitante.flagChargebackPicpay': 'is_chargeback_pp',
        'solicitante.atrasos.qtdDiasAtrasoP2PL': 'max_dpd_p2p',
        'solicitante.atrasos.qtdDiasAtrasoBNPLMova': 'max_dpd_bnpl_mova',
        'solicitante.statusCPF': 'status_cpf',
        'solicitante.flagReneg': 'is_reneg_empp',
        'solicitante.flagFNV': 'is_fnv_empp',
        'solicitante.restritivosBVS.vlrMaxRestritivosBVSM1': 'max_rest_value_bvs_m1',
        'solicitante.restritivosBVS.vlrMaxRestritivosBVSM2': 'max_rest_value_bvs_m2',
        'solicitante.restritivosBVS.vlrMaxRestritivosBVSM3': 'max_rest_value_bvs_m3',
        'solicitante.vlrMaxRestritivosSerasa90Dias': 'value_rest_serasa',
        'solicitante.idade': 'age',
        'solicitante.atrasos.qtdDiasAtrasoParcelados': 'max_dpd_installments',
        'solicitante.atrasos.qtdDiasAtrasoCartao': 'max_dpd_card',
        'solicitante.rendaLiquidaPicpay': 'income_picpay',
        'solicitante.flagLimiteGarantidoContratado': 'has_guaranteed_limit',
        'solicitante.limites.flagCartaoCreditoContratado': 'is_credit',
        'solicitante.flagCartaoDebitoContratado': 'is_debit',
        'solicitante.scoreInternoLongo': 'credit_appcard_int30m12_v01',
        'solicitante.scoreInterno': 'credit_appcard_blend30m3_v01',
        'solicitante.scoreExterno': 'credit_appcard_ext30m3_v01',
        'solicitante.scoreAppSerasa': 'score_appl_serasa',
        'solicitante.scoreInternoCurto': 'credit_appcard_int30m3_v01',
        'solicitante.investimentoTotal': 'principality_total_investment',
        'solicitante.flagInvestimentoPrincipalidade': 'has_principality_investment',
        'solicitante.fonteRenda': 'source_income',
        'solicitante.dataRegistroPrimeiroContrato': 'original_registered_date',
        'solicitante.saldoMinimo30d': 'balance_min_30d',
        'solicitante.flagMAT30d': 'is_mat_30',
        'solicitante.flagMAT60d': 'is_mat_60',
        'solicitante.flagMAT90d': 'is_mat_90',
        'solicitante.flagPortabilidade': 'is_portability',
        'solicitante.saldoMedio': 'average_balance',
        'solicitante.flagPrincipalidadeAll': 'principality_all',
        'solicitante.flagCCRot': 'flag_cc_rot',
        'solicitante.mediaSaldoTotal': 'mean_total_balance_d0',
        'solicitante.dataRegistroPrimeiroContrato': 'registered_at',
        'solicitante.flagFuncionario': 'is_employee',
        'solicitante.flagPrincipalidade': 'is_principality_all',
        'solicitante.mesFlagPrincipalidade': 'principality_in_months',
        'solicitante.dataAdmissaoFopag': 'fopag_dt_admissao',
        'flagSmallLimits': 'is_small_limits_cp',
        'solicitante.idade': 'idade',
        'solicitante.numeroDocumento': 'cpf',
        'solicitante.qtdIF': 'number_of_institutions',
        'solicitante.consumerID': 'consumer_id',
        }, inplace=True)

In [ ]:
cenariosTesteIntegradoRenomeado

,cpf,consumer_id,etapa,number_of_institutions,source_income,has_invalid_contact_information,has_denied_registration,has_ongoing_registration,has_deactivated_account,is_picpay_test_account,...,flag_cc_rot,has_principality_investment,principality_total_investment,solicitante.saldoInvestimento,solicitante.listaRestritivosSerasa.0.tipo,solicitante.listaRestritivosSerasa.0.montanteTotal,solicitante.listaRestritivosSerasa.0.valorTotal,Debug - ResultadoEsperado,Debug - Cenario,Debug - idSolicitacao Massa
0,12345678909,101010101,,1,01. Fopag,0,0,0,0,0,...,0,1,0,0,N,0,0,APROVADO,Hard Filter Flexibilizado,0
1,12345678909,10001,,1,01. Fopag,0,0,0,0,0,...,0,1,0,0,N,0,0,NEGADO,Hard Filter Flexibilizado,7
2,12345678909,101010101,,1,01. Fopag,0,0,0,0,0,...,0,1,0,0,N,0,0,APROVADO,Filtros Politica,0
3,12345678909,1,,1,01. Fopag,0,0,0,0,0,...,0,1,0,0,N,0,0,NEGADO,Filtros Politica,22
4,12345678909,101010101,,1,01. Fopag,0,0,0,0,0,...,0,1,0,0,N,0,0,APROVADO,Bacen DCO Flexibilizado,0
5,12345678909,10,,1,01. Fopag,0,0,0,0,0,...,0,1,0,0,N,0,0,NEGADO,Bacen DCO Flexibilizado,194
6,12345678909,101010101,,1,01. Fopag,0,0,0,0,0,...,0,1,0,0,N,0,0,APROVADO,Hard Filter Nao Flexibilizado,0
7,12345678909,10001,,1,01. Fopag,0,0,0,0,0,...,0,1,0,0,N,0,0,NEGADO,Hard Filter Nao Flexibilizado,8
8,12345678909,101010101,,1,01. Fopag,0,0,0,0,0,...,0,1,0,0,N,0,0,APROVADO,Politica Concessao + Bacen Sem Flexibilizacao,0
9,12345678909,9,,1,01. Fopag,0,0,0,0,0,...,0,1,0,0,N,0,0,NEGADO,Politica Concessao + Bacen Sem Flexibilizacao,75


In [ ]:
idSolcitacao = 
for payload in listaResultadoPythonBureauCadastral:
    if (payload['payloadHomol']['saidas']['statusDecisao'] == 'APROVADO'):
        print(payload)
        break

###### Export csv dos arquivos

In [ ]:
listacustomer_status = ['id',
                        'status_cpf',
                        'birth_date',
                        'age',
                        'digital_account_status',
                        'is_fraud_pp',
                        'has_restriction_annotation',
                        'update_at',
                        'range_intern_score_card',
                        'new_group_card',
                        'is_approved_small_limits_card',
                        'ds_strategy_decision_public_cp',
                        'dt_abt_reference_date_cp',
                        'qt_credit_installment_max_cp',
                        'ds_swap_set_picpay_cp',
                        'has_guaranteed_limit',
                        'is_credit',
                        'is_debit',
                        'credit_appcard_int30m12_v01',
                        'credit_appcard_blend30m3_v01',
                        'credit_appcard_ext30m3_v01',
                        'score_appl_bvs',
                        'score_appl_serasa',
                        'credit_appcard_int30m3_v01',
                        'has_principality',
                        'is_mat',
                        'is_mau',
                        'principality_total_investment',
                        'has_principality_investment',
                        'source_income',
                        'income_picpay',
                        'gross_income_picpay',
                        'original_registered_date',
                        'balance_min_30d',
                        'is_mat_30',
                        'balance_min_60d',
                        'is_mat_60',
                        'balance_min_90d',
                        'is_mat_90',
                        'is_portability',
                        'employee_status',
                        'partition_date',
                        'ds_strategy_decision_public',
                        'ds_strategy_decision',
                        'average_balance',
                        'income',
                        'principality_all',
                        'flag_cc_rot',
                        'sld_tomado_cc',
                        'lim_disp_cc',
                        'mean_total_balance_d0',
                        'registered_at',
                        'is_employee',
                        'is_principality_all',
                        'principality_in_months',
                        'fopag_dt_admissao',
                        'is_small_limits_cp',
                        'Debug - ResultadoEsperado',
                        'Debug - Cenario',
                        'Debug - idSolicitacao Massa'
                       ]

listacredit_policy_hard_filters = ['id',
                                   'consumer_id',
                                   'has_invalid_contact_information',
                                   'has_outdated_os',
                                   'has_deactivated_account',
                                   'is_picpay_test_account',
                                   'has_denied_registration',
                                   'has_ongoing_registration',
                                   'is_user_fraud_pp',
                                   'is_ppcard_fraud',
                                   'is_dead_user_pp',
                                   'is_in_blck_ppcard_reneg',
                                   'is_on_hold_forced',
                                   'is_chargeback_pp',
                                   'max_dpd_p2p',
                                   'max_dpd_bnpl_mova',
                                   'status_cpf',
                                   'is_cpf_archived',
                                   'is_reneg_empp',
                                   'is_fnv_empp',
                                   'information_year_month',
                                   'max_rest_value_bvs_m1',
                                   'max_rest_value_bvs_m2',
                                   'max_rest_value_bvs_m3',
                                   'value_rest_serasa',
                                   'age',
                                   'max_dpd_installments',
                                   'max_dpd_card',
                                   'income_picpay',
                                   'Debug - ResultadoEsperado',
                                   'Debug - Cenario',
                                   'Debug - idSolicitacao Massa'
                                  ]

listamodel_income_nrt = ['id',
                         'consumer_id',
                         'dt_ref_sample',
                         'serasa_custom_income',
                         'in_mau35',
                         'in_mat35',
                         'in_mat3m',
                         'relationship_time',
                         'is_emergency_aid',
                         'is_public_servant',
                         'median_4m_wallet',
                         'median_2m_bills',
                         'bvs_income',
                         'is_black_infinite_card',
                         'lb_devos_r00_reg',
                         'median_cashin',
                         'median_tpv',
                         'median_qtd_transactions',
                         'region_cpf_basic_register',
                         'scr_est_income',
                         'bills_est_income',
                         'fl_consent_of',
                         'idade',
                         'modelo_transacional_3v0',
                         'median_4m_bills_opf',
                         'opf_income',
                         'is_variante_cartao',
                         'Debug - ResultadoEsperado',
                         'Debug - Cenario',
                         'Debug - idSolicitacao Massa'
                        ]

listamodel_nrt_scr_application = ['id',
                                  'consumer_id',
                                  'referenced_date',
                                  'score_int_final_al',
                                  'score_scr_final_al',
                                  'score_serasa',
                                  'blend_intextscr_al',
                                  'gh_intextscr',
                                  'blend_scrext_al',
                                  'gh_scrext',
                                  'blend_intext_al',
                                  'gh_intext',
                                  'score_serasa_al',
                                  'gh_serasa',
                                  'origem_blend',
                                  'blend_final_al',
                                  'gh_final',
                                  'month',
                                  'year',
                                  'Debug - ResultadoEsperado',
                                  'Debug - Cenario',
                                  'Debug - idSolicitacao Massa'
                                 ]

listascr_bacen = [
                'id',
                'cpf',
                'reference_date',
                'modality_code',
                'assumed_coobligation',
                'received_coobligation',
                'start_relationship_date',
                'number_of_institutions',
                'exchange_variation',
                'addressing_collection',
                'percentage_documents_processed',
                'processed_volume_percentage',
                'number_of_operations_discordance',
                'number_of_subjudice_operations',
                'number_of_operations',
                'full_responsibility_disagreement',
                'full_responsibility_subjudice',
                'vendor_indirect_risk',
                'consumer_type',
                'v_20',
                'v_40',
                'v_60',
                'v_80',
                'v_110',
                'v_120',
                'v_130',
                'v_140',
                'v_150',
                'v_160',
                'v_165',
                'v_170',
                'v_175',
                'v_180',
                'v_190',
                'v_199',
                'v_205',
                'v_210',
                'v_220',
                'v_230',
                'v_240',
                'v_245',
                'v_250',
                'v_255',
                'v_260',
                'v_270',
                'v_280',
                'v_290',
                'v_310',
                'v_320',
                'v_330',
                'generation_date',    
                'Debug - ResultadoEsperado',
                'Debug - Cenario',
                'Debug - idSolicitacao Massa'
]

# Gera um compilado das colunas existentes para verificar se estamos abrangendo todas nos arquivos
colunasSeparadas = listacustomer_status + listacredit_policy_hard_filters + listamodel_income_nrt + listamodel_nrt_scr_application + listascr_bacen
allColumns = cenariosTesteIntegradoRenomeado.columns.tolist()

for col in allColumns:
    if col not in colunasSeparadas:

        print("Verificar o arquivo de destino para: ",col)



colunasExistentes = [col for col in listacustomer_status if col in cenariosTesteIntegradoRenomeado.columns]
cenarioscustomer_status = cenariosTesteIntegradoRenomeado[colunasExistentes]
#cenarioscustomer_status.to_csv(r'/massas/cenarios_customer_status.csv',index=False)

colunasExistentes = [col for col in listacredit_policy_hard_filters if col in cenariosTesteIntegradoRenomeado.columns]
cenarioscredit_policy_hard_filters = cenariosTesteIntegradoRenomeado[colunasExistentes]
#cenarioscredit_policy_hard_filters.to_csv(r'/massas/cenarios_credit_policy_hard_filters.csv',index=False)

colunasExistentes = [col for col in listamodel_income_nrt if col in cenariosTesteIntegradoRenomeado.columns]
cenariosmodelincomenrt = cenariosTesteIntegradoRenomeado[colunasExistentes]
#cenariosmodelincomenrt.to_csv(r'/massas/cenarios_model_income_nrt.csv',index=False)

colunasExistentes = [col for col in listamodel_nrt_scr_application if col in cenariosTesteIntegradoRenomeado.columns]
cenariosmodelnrtscrapplication = cenariosTesteIntegradoRenomeado[colunasExistentes]
#cenariosmodelnrtscrapplication.to_csv(r'/massas/cenariosmodelnrtscrapplication.csv',index=False)

colunasExistentes = [col for col in listascr_bacen if col in cenariosTesteIntegradoRenomeado.columns]
cenarioslistascrbacen = cenariosTesteIntegradoRenomeado[colunasExistentes]


Verificar o arquivo de destino para:  etapa
Verificar o arquivo de destino para:  solicitante.flagSmallLimits
Verificar o arquivo de destino para:  solicitante.flagDesenrola
Verificar o arquivo de destino para:  solicitante.rendaLiquidaPicpayBatch
Verificar o arquivo de destino para:  solicitante.credilink.situacaoDocumento
Verificar o arquivo de destino para:  solicitante.credilink.obito
Verificar o arquivo de destino para:  marcacaoMesa
Verificar o arquivo de destino para:  flagErroAurora
Verificar o arquivo de destino para:  flagErroVisaoCliente
Verificar o arquivo de destino para:  flagErroBacen
Verificar o arquivo de destino para:  flagErroCredilink
Verificar o arquivo de destino para:  flagErroReceitaFederal
Verificar o arquivo de destino para:  flagErroSerasa
Verificar o arquivo de destino para:  solicitante.limites.limiteDisponivel
Verificar o arquivo de destino para:  solicitante.limites.valorContratadoCartao
Verificar o arquivo de destino para:  solicitante.maxDiasAtraso
Veri